In [ ]:
import torch
import torchvision
import tensorflow as tf
tf.config.list_physical_devices("GPU")
print(torch.__version__)
print(tf.__version__)
print(torch.cuda.get_arch_list())
print(torch.version.cuda)

In [ ]:
#print(pd.__version__) 1.4.1
#print(np.__version__) 1.21.2
#print(matplotlib.__version__) 3.5.1
#print(sklearn.__version__) 1.0.2
#print(mne.__version__) 0.24.1
#'Corrections, to_data_frame, get_epoch , no of output neuron'

In [ ]:
data_file_path = './patient_info_txt/data_files_'
csv_file_path = './patient_info_txt/Invasive_data_'
patient_id = [115,139,253,264,442,583,620,635,862,958,1146]
p_id = 2 
interval = 2
test_list = [80] #7[12,14] #8[106,109] #9[80,81]
soz_channel_list = {                      #seizure
    115 : ["HR3","HR4","HR5","HR6"],      #26
    139 : ["HL2","HL4","HL3","TBA1"],     #6
    253 : ["HRB2","HRC2","HRB3","HRC3"],  #7  
    264 : ["BLA1","BLC1","BRA1","TRA3"],  #8   
    442 : ["HRA4","TBA1","HRA5","TBA2"],  #22  
    583 : ["TLB1","TLA1","HL1","TBA1"],   #23  
    620 : ["TLA1","TLB2","TLB3","TLA2"],  #7   yes
    635 : ["HL1","HL9","HRA1","HRA2"],     #21  yes
    862 : ["IHB2","IHB3","IHB1","GC7"],   #9    yes
    958 : ["GE3","GH6","OPL5","TBB2"],    #16  yes
    1146 : ["ICL1","SCL7","SCL8","SCR5"], #26
}
print(soz_channel_list[patient_id[p_id]], [patient_id[p_id]], test_list)
patient_channels = soz_channel_list[patient_id[p_id]]
#index for function - 2

In [ ]:
import os
import copy
import pandas as pd
import numpy as np
import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import sklearn
import mne
from mne.time_frequency import psd_welch
import csv
import datetime
from datetime import timedelta
datetimeFormat = '%Y-%m-%d %H:%M:%S.%f'
import re
import imblearn

from numpy.fft import fft, fftfreq
from scipy import signal
from mne.time_frequency import tfr_morlet

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
%matplotlib notebook
mne.set_log_level('ERROR') 


#path_to_data = './patient_info_txt/data_files_442.txt'
#path_to_csv = './patient_info_txt/Invasive_data_442.csv'

path_to_data = data_file_path + str(patient_id[p_id]) + '.txt'
path_to_csv = csv_file_path + str(patient_id[p_id]) + '.csv'

data_dict = {}

def square_data(inputs):
    data_df = pd.Series(inputs)
    data_df_sq = data_df.pow(2)
    data_df_np = data_df_sq.to_numpy()
    return data_df_np

def normalize_data(inputs):
    data_df_HRC1 = pd.Series(inputs)
    data_df_std_dev = data_df_HRC1.rolling(time_steps).std() # 30 seconds
    data_df_norm = data_df_HRC1.divide(data_df_std_dev)  
    data_df_norm[:time_steps-1].update(data_df_HRC1[:time_steps-1]) # if NaN, norm = original value
    data_df_HRC1_np = data_df_norm.to_numpy()
    return data_df_HRC1_np

def diff_data(inputs):  #TODO : Do this before rereferencing
    data_df = pd.Series(inputs)
    data_df_diff= data_df.diff()
    data_df_np = data_df_diff.to_numpy()
    return data_df_np

def dur(num):
    if num/60 > 1:
        return 60.0
    else:
        return float(num)

#Reref    
#HRA = ['HRA1','HRA2','HRA3','HRA4','HRA5']
#HRB = ['HRB1','HRB2','HRB3','HRB4','HRB5','HRC1','HRC2','HRC3','HRC4','HRC5']
#HRC = ['HRB1','HRB2','HRB3','HRB4','HRB5','HRC1','HRC2','HRC3','HRC4','HRC5']

if torch.cuda.is_available():
    print('CUDA-enabled GPU found. Training should be faster.')
else:
    print('No GPU found. Training will be slow')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
my_database ={'Patient 1':{'PatientID': 0, 'FileID': [], 'Frequency': 512, 'Duration': 0}}

print('HERE WE GO!!!!!!!!!!!!!!')
with open(path_to_data) as file:
    lines = file.readlines()
    lines_iter = iter(lines)
    lines = [line.rstrip() for line in lines]
    #print('lines', lines, len(lines))
            
def load_raw_dataset(line, my_database):
    """Load a recording from the iEEG dataset.
    
    Parameters
    ----------
    raw_fname : str
        Path to the .data file containing the raw data.
    
    Returns
    -------
    mne.io.Raw :
        Raw object containing the EEG. 
    """
    #for line in range(0,len(lines)):
    print(line, type(line))
    #if not line.startswith("#"):
    entry = 'EEG_inv_' + str(line)[-18:-15] + '_' + str(line)[-9:-5]
    print('Value for entry',entry)
    my_database['Patient 1']['PatientID'] = str(line)[-18:-15]
    my_database['Patient 1']['FileID'] = str(line)[-9:-5]
    print(my_database)
    data_raw =  mne.io.read_raw_nicolet(str(line), 'eeg', preload=True)
    
    subj_num, rec_num = int(str(line)[-18:-15]), int(str(line)[-9:-5])
    print('subj_num',subj_num ,'rec_num',rec_num)
    data_raw.info['subject_info'] = {'id': subj_num, 'rec_id': rec_num}
    return data_raw

# Check 
#load_raw_dataset(lines,my_database)
# Load recordings
data_raws = [load_raw_dataset(line, my_database) for line in lines]
print('Database entries',len(data_raws) , type(data_raws) )

#for idx in range(0,len(data_raws)):
#    print(data_raws[idx].info)
#    print(data_raws[idx].plot())

In [ ]:
#data_raws[1].plot()
#data_raws[6].plot()
#data_raws[8].plot()
#data_raws[10].plot()
# Add annotation to data - onset, duration, description (in s), orig_time 0 meas_date
'''
my_annot = mne.Annotations(onset= 22*60,  # in seconds
                           duration= 21,  # in seconds, too
                           description='True Prediction')
print(my_annot)
data_raws[1].set_annotations(my_annot)
print(data_raws[1].annotations)
data_raws[1].plot()'''

my_annot = mne.Annotations(onset= 1*60,  # in seconds
                           duration= 80,  # in seconds, too
                           description='True Prediction')
print(my_annot)
data_raws[15].set_annotations(my_annot)
print(data_raws[15].annotations)
data_raws[15].plot()


In [ ]:
# Set reference channel for all data points for all channels individually
def set_reference(data_raw, p_id):
    data_array = []
    sfreq = data_raw.info['sfreq']
    print('sfreq', sfreq)
    print('4 electrodes for the patient',soz_channel_list[patient_id[p_id]])
    patient_channels = soz_channel_list[patient_id[p_id]]
    print('patient channels',patient_channels)
    all_channels = data_raw.ch_names
    #print('all channels',all_channels)
    for item in patient_channels:
        #print('For channel', item)
        ch_group = " ".join(re.findall("[a-zA-Z]+", item))
        ch_group = str(ch_group)
        ch_group = [x for x in all_channels if re.search(ch_group, x)]
        #print('Electrode group', str(ch_group))
        data_ch_dict = data_raw.copy()
        data_ch_dict = data_ch_dict.pick_channels(ch_names= ch_group, ordered=False)
        data_ch_dict.set_eeg_reference(ref_channels= ch_group)
        data_ch_dict = data_ch_dict.pick_channels(ch_names= [item], ordered=False)
        df_channel = data_ch_dict.to_data_frame()
        #print('Each channel', df_channel)
        #print('Info', data_ch_dict.info)
        df_channel_array = np.array(df_channel[item])
        #print(df_channel_array, df_channel_array.shape)
        data_array.append(df_channel_array)
        
    reref_data = np.array(data_array)
    print('reref data', reref_data, reref_data.shape)
    info = mne.create_info(ch_names= patient_channels, sfreq=sfreq, ch_types = ['eeg']* 4)
    print('Assert',len(reref_data), len(info['ch_names']), patient_channels)
    simulated_raw = mne.io.RawArray(reref_data, info)
    #print(simulated_raw.info)
    return simulated_raw 

reref_raws = []
for idx in range(0,len(data_raws)):
    #print('BEFORE RE-REFERENCING',data_raws[idx].info)
    #data_raws[idx].plot()
    reref_raws.append(set_reference(data_raws[idx],p_id))
    #reref_raws[idx] = set_reference(data_raws[idx],2)
    #print('AFTER RE-REFERENCING',reref_raws[idx].info)
    #reref_raws[idx].plot()
    #break

print(reref_raws)

In [ ]:
# use notch filter 
#https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.notch_filter
for idx in range(0,len(reref_raws)):
    print(reref_raws[idx].info)
    break
print(interval)

In [ ]:
def dt_(t_sample, meas_data):
    #time = float(datetime.datetime.strptime(t_sample, datetimeFormat))
    time = float((datetime.datetime.strptime(t_sample, datetimeFormat)- 
                 datetime.datetime.strptime(meas_data, datetimeFormat)).seconds)
    return time

In [ ]:
def process_file(data_raw, csv_row, interval ):
    df = pd.read_csv(path_to_csv, usecols=['meas_date','onset', 'offset','onset_sample','offset_sample', 'channels'])
    cnt_data =0
    cnt_data_ch = 0
    columns_ = []
    meas_date = []
    seizure_count = []
    data_ch_dict = [None] * len(data_raw)
    data_epoch = [None] * len(data_raw)
    #print('len of data_raw', len(data_raw))
    
    #Generate information on number of seizures in a file
    all_meas_data = df.iloc[:,0]
    all_onset = df.iloc[:,1]
    all_offset = df.iloc[:,2]
    all_onset_sample = df.iloc[:,3]
    all_offset_sample = df.iloc[:,4]
    meas_date = all_meas_data.tolist()
    onset_date = all_onset.tolist()
    offset_date = all_offset.tolist()
    onset_sample_date = all_onset_sample.tolist()
    offset_sample_date = all_offset_sample.tolist()
    for i in range(0,len(meas_date)):
        if i==0:
            seiz = 1 ; cnt =1
        else:
            if (meas_date[i-1] == meas_date[i]):
                cnt +=1 ; seiz = cnt
            else:
                seiz = 1 ; cnt =1
        seizure_count.append(seiz)
    #print('Seizure count list',seizure_count, len(seizure_count), csv_row)
    
    num_of_seizure_in_file = [1] * len(seizure_count)
    for i in range(0,len(seizure_count)): 
        if seizure_count[i] == 2:
            num_of_seizure_in_file[i] = 2
            num_of_seizure_in_file[i-1] = 2
        if seizure_count[i] == 3:
            num_of_seizure_in_file[i] = 3
            num_of_seizure_in_file[i-1] = 3
            num_of_seizure_in_file[i-2] = 3
    #print('num of seizures from a file',num_of_seizure_in_file)
    
    #print("seizure starts after:",tdiff_on, "seizure lasts for",tdur_seiz,"stops after",tdiff_off)
    #print('Seizure onset',onset,'Seizure offset',offset )
    #print(print("seizure starts after:",tdiff_on.seconds, "seizure lasts for",tdur_seiz.seconds,"stops after",tdiff_off.seconds))

    '''
    for arr in range(len(channel)):
        for lis in range(len(channel[arr])):
            channel[arr][lis].replace('\'','')
            channel[arr][lis].replace(']','')
            channel[arr][lis].replace('[','')
            channel[arr][lis].replace(' ','')
    #print("Shape and type of tdiff_on:",type(tdiff_on),tdiff_on, 'channel',channel)
    
    if channel.find(',') != -1:
        channel = list(channel.split(","))
        #rint('HEY FOUND A ,', channel,len(channel), type(channel))
    else:
        channel = list(channel.split(","))
        #print('HEY DID NOT FIND ,', channel, len(channel), type(channel))
    '''
    cnt_data += 1
    
    #Signal prepreocessing
    #print('measurement data',meas_data, meas_data_next)
    
    #ch_name = entry 
    #ref_ch = HRB
    #ref_ch = data_raw.ch_names

    data_ch_dict = data_raw.copy()
    #print('Before re-referencing',data_ch_dict.info)
    #data_ch_dict.plot()
    #data_ch_dict = data_ch_dict.pick_channels(ch_names= ref_ch, ordered=False)
    #data_ch_dict.set_eeg_reference(ref_channels=ref_ch) #TODO add rereferencing
    #print('After re-referencing',data_ch_dict.info)
    #data_ch_dict.plot()
    
    #Resampling and anti-aliasing low pass filter
    data_ch_dict.load_data().filter(l_freq=None, h_freq=90, method='fir',picks=patient_channels)
    #print('AFTER ANTI ALISAING FILTER')
    #data_ch_dict.plot_psd(picks=patient_channels)
    if data_ch_dict.info['sfreq'] != 256.0:
        data_ch_dict = data_ch_dict.resample(256)
    #data_ch_dict.plot()
    
    #print('After resampling',data_ch_dict.info)
    data_ch_dict.load_data().filter(l_freq=0.2, h_freq=48, method='fir',picks=patient_channels)
    #print('After filtering',data_ch_dict.info)
    #print('AFTER BPF')
    #data_ch_dict.plot_psd(picks=patient_channels)
    
    do_normalization = False
    calculate_dt= False
    #time_steps = (10*60)/2
    time_steps = 10
    
    #Calculate time difference     #before re-referencing TODO
    if calculate_dt == True:
        #print('First derivative', channel)
        data_ch_dict.apply_function(diff_data, picks=channel)
        #print('After first derivative',data_ch_dict.info)
        #data_ch_dict.plot()
    else:
        data_ch_dict = data_ch_dict
        #print('First derivative not taken')
    
    # TODO ; Temporary use HRB2 and HRC2
    #channel_temp = ['HRB2','HRC2', 'HRB3','HRC3']
    #channel_temp = ['HRB2','HRC2']
    #channel_temp = ['TLB1', 'TLA1', 'HL1', 'TBA1']
    #channel_temp = ['HRA4', 'TBA1', 'HRA5', 'TBA2']
    #data_ch_dict.pick_channels(ch_names = channel_temp, ordered = False)
    #print(data_ch_dict.info)
    
    #seizure starts after: 17 seizure lasts for 55 stops after 73
    #print(print("seizure starts after:",tdiff_on.seconds, "seizure lasts for",tdur_seiz.seconds,"stops after",tdiff_off.seconds))
    #print(print("seizure starts after:",tdiff_on.microseconds, "seizure lasts for",tdur_seiz.microseconds,"stops after",tdiff_off.microseconds))
    #seizure starts after: 0:00:17.558594 seizure lasts for 0:00:55.945312 stops after 0:01:13.503906

    #Generate labels
    for col in range(0,len(df.columns)):
        data = df.iloc[csv_row,col]
        columns_.append(data)
    
    meas_data = columns_[0]
    onset = columns_[1]
    offset = columns_[2]
    onset_sample = columns_[3]
    offset_sample = columns_[4]
    channel = columns_[5]

    tdur_seiz = datetime.datetime.strptime(offset, datetimeFormat)- datetime.datetime.strptime(onset, datetimeFormat)
    tdiff_on = datetime.datetime.strptime(onset, datetimeFormat)- datetime.datetime.strptime(meas_data, datetimeFormat)
    tdiff_off = datetime.datetime.strptime(offset, datetimeFormat)- datetime.datetime.strptime(meas_data, datetimeFormat)
    
    '''
    if (csv_row == len(seizure_count)-1):
        print('This file has ',seizure_count[csv_row],'seizures')
        
    elif(csv_row == len(seizure_count)-2):
        if(seizure_count[csv_row] == 1 and seizure_count[csv_row+1] == 2 ):
            print('This file has 2 seizures')
        elif(seizure_count[csv_row] == 2 and seizure_count[csv_row+1] == 3 ):
            print('This file has 3 seizures')
        else:
            print('This file has 1 seizure')
        
    elif ((seizure_count[csv_row] == 1) and (seizure_count[csv_row+1] ==1)):
        print('This file has only one seizure')
    
    elif(((seizure_count[csv_row] == 1) and (seizure_count[csv_row+1] ==2) and (seizure_count[csv_row+2] ==1)) or ((seizure_count[csv_row] ==2) and (seizure_count[csv_row+1] ==1))):
        print('This file has 2 seizures')
    
    elif(((seizure_count[csv_row] == 1) and (seizure_count[csv_row+1] ==2) and (seizure_count[csv_row+2] ==3)) or ((seizure_count[csv_row] == 2) and (seizure_count[csv_row+1] ==3)) or (seizure_count[csv_row] == 3)):
        print('This file has 3 seizures')'''
    
    #Extract 5s epochs : Create epochs and labels
    interval = interval  #TODO 2 sec
    time_window = interval * data_ch_dict.info['sfreq'] # 5*512
    #print('time window', time_window)
    
    #Split
    if (num_of_seizure_in_file[csv_row] == 1):
        data_ch_dict = data_ch_dict.copy()
        print('data_ch_dict_1',data_ch_dict)
        df = data_ch_dict.to_data_frame()
        df['time_s'] = df['time'] / 1000.0
        df['target'] = 0
        df['target'] = np.where(((df.time_s >= float(tdiff_on.seconds)) & (df.time_s <= float(tdiff_off.seconds))),1 ,df.target)        
        seizure_length = float(tdiff_off.seconds) - float(tdiff_on.seconds)
        print('Length of seizure is', seizure_length, 'seconds' )
        
    elif (num_of_seizure_in_file[csv_row] == 2):
        if (seizure_count[csv_row] ==1):
            split_1 = dt_(offset_date[csv_row],meas_date[csv_row])  +((dt_(onset_date[csv_row+1],meas_date[csv_row]) - dt_(offset_date[csv_row],meas_date[csv_row]))/2 )
            data_ch_dict_2_1 = data_ch_dict.copy().crop(tmin = 0, tmax = split_1)
            data_ch_dict = data_ch_dict_2_1.copy()
            print('data_ch_dict_2_1',data_ch_dict)
            df = data_ch_dict.to_data_frame()
            df['time_s'] = df['time'] / 1000.0
            df['target'] = 0
            df['target'] = np.where(((df.time_s >=float(tdiff_on.seconds)) & (df.time_s <= float(tdiff_off.seconds))),1,df.target)
            print('2_1', split_1,float(tdiff_on.seconds), float(tdiff_off.seconds))
            seizure_length = float(tdiff_off.seconds) - float(tdiff_on.seconds)
            print('Length of seizure is', seizure_length, 'seconds' )
            
        if (seizure_count[csv_row] ==2):
            split_1 = dt_(offset_date[csv_row-1],meas_date[csv_row])  +((dt_(onset_date[csv_row],meas_date[csv_row]) - dt_(offset_date[csv_row-1],meas_date[csv_row]))/2 )
            start = (dt_(onset_date[csv_row],meas_date[csv_row]) - dt_(offset_date[csv_row-1],meas_date[csv_row]))/2
            end = start + (dt_(offset_date[csv_row],meas_date[csv_row]) - dt_(onset_date[csv_row],meas_date[csv_row]))
            data_ch_dict_2_2 = data_ch_dict.copy().crop(tmin= split_1)
            data_ch_dict = data_ch_dict_2_2.copy()
            print('data_ch_dict_2_2',data_ch_dict)
            df = data_ch_dict.to_data_frame()
            df['time_s'] = df['time'] / 1000.0
            df['target'] = 0
            df['target'] = np.where(((df.time_s >= start) & (df.time_s <= end)),1,df.target)
            print('2_2', split_1, start, end)
            seizure_length = end - start
            print('Length of seizure is', seizure_length, 'seconds' )
            
    elif (num_of_seizure_in_file[csv_row] == 3):
        if ( seizure_count[csv_row] ==1):
            split_2 = dt_(offset_date[csv_row],meas_date[csv_row])  +((dt_(onset_date[csv_row+1],meas_date[csv_row]) - dt_(offset_date[csv_row],meas_date[csv_row]))/2 )
            data_ch_dict_3_1 = data_ch_dict.copy().crop(tmin=0 ,tmax= split_2)
            data_ch_dict = data_ch_dict_3_1.copy()
            print('data_ch_dict_3_1',data_ch_dict)
            df = data_ch_dict.to_data_frame()
            df['time_s'] = df['time'] / 1000.0
            df['target'] = 0
            df['target'] = np.where(((df.time_s >= float(tdiff_on.seconds)) & (df.time_s <= float(tdiff_off.seconds))),1,df.target)
            print('3_1', split_2, float(tdiff_on.seconds), float(tdiff_off.seconds))
            seizure_length = float(tdiff_off.seconds) - float(tdiff_on.seconds)
            print('Length of seizure is', seizure_length, 'seconds' )
            
        if ( seizure_count[csv_row] ==2):
            split_2 = dt_(offset_date[csv_row-1],meas_date[csv_row])  +((dt_(onset_date[csv_row],meas_date[csv_row]) - dt_(offset_date[csv_row-1],meas_date[csv_row]))/2 )
            split_3 = dt_(offset_date[csv_row],meas_date[csv_row])  +((dt_(onset_date[csv_row+1],meas_date[csv_row]) - dt_(offset_date[csv_row],meas_date[csv_row]))/2 )
            start = (dt_(onset_date[csv_row],meas_date[csv_row]) - dt_(offset_date[csv_row-1],meas_date[csv_row]))/2
            end = start + (dt_(offset_date[csv_row],meas_date[csv_row]) - dt_(onset_date[csv_row],meas_date[csv_row])) 
            data_ch_dict_3_2 = data_ch_dict.copy().crop(tmin=split_2 ,tmax=split_3)
            data_ch_dict = data_ch_dict_3_2.copy()
            print('data_ch_dict_3_2',data_ch_dict)
            df = data_ch_dict.to_data_frame()
            df['time_s'] = df['time'] / 1000.0
            df['target'] = 0
            df['target'] = np.where(((df.time_s >= start) & (df.time_s <= end)),1,df.target)
            print('3_2', split_2, split_3, start, end)
            seizure_length = end - start
            print('Length of seizure is', seizure_length, 'seconds' )
            
        if ( seizure_count[csv_row]==3):
            split_3 = dt_(offset_date[csv_row-1],meas_date[csv_row])  +((dt_(onset_date[csv_row],meas_date[csv_row]) - dt_(offset_date[csv_row-1],meas_date[csv_row]))/2 )
            start = (dt_(onset_date[csv_row],meas_date[csv_row]) - dt_(offset_date[csv_row-1],meas_date[csv_row]))/2
            end = start + (dt_(offset_date[csv_row],meas_date[csv_row]) - dt_(onset_date[csv_row],meas_date[csv_row]))
            data_ch_dict_3_3 = data_ch_dict.copy().crop(tmin=split_3 )
            data_ch_dict = data_ch_dict_3_3.copy()
            print('data_ch_dict_3_3',data_ch_dict)
            df = data_ch_dict.to_data_frame()
            df['time_s'] = df['time'] / 1000.0
            df['target'] = 0
            df['target'] = np.where(((df.time_s >= start) & (df.time_s <= end)),1,df.target)
            print('3_3', split_3, start, end)
            seizure_length = end - start
            print('Length of seizure is', seizure_length, 'seconds' )
    
     # Store data, time and label as dataframe
    #df = data_ch_dict.to_data_frame()
    #print(df)
    #df['time_s'] = df['time'] / 1000.0
    #df['target'] = 0
    #df['target'] = np.where(((df.time_s >= float(tdiff_on.seconds)) & (df.time_s <= float(tdiff_off.seconds))),1,df.target)
    #print(df) 
    
    '''
    #Crop the data to include less non-ictal activity : Temporary logic
    def define_df(data_ch_dict, idx):
        df1 = data_ch_dict.to_data_frame()
        df1['time_s'] = df1['time'] / 1000.0
        df1['target'] = 0
        df1['target'] = np.where(( (df1.time_s+ idx >= float(tdiff_on.seconds)) & (df1.time_s + idx <= float(tdiff_off.seconds))),1,df1.target)
        return df1
    if (tdiff_on.seconds < 1200):
        data_ch_dict_n = data_ch_dict.copy().crop(tmin=0,tmax=1200) #0,1200
        print('Seizure occurs in first half')
        df_subset = define_df(data_ch_dict_n, 0)
    elif (tdiff_on.seconds > 2400):
        data_ch_dict_n = data_ch_dict.copy().crop(tmin=2400, tmax =3599) # 2400,3600
        print('Seizure occurs in end')
        df_subset = define_df(data_ch_dict_n, 2400)
    else:
        data_ch_dict_n = data_ch_dict.copy().crop(tmin=1200,tmax=2400) # 1200,2400
        print('Seizure occurs in mid ')
        df_subset = define_df(data_ch_dict_n, 1200)
       #'''    
    
    data_epoch = mne.make_fixed_length_epochs(data_ch_dict, duration= interval, preload=True, id= csv_row) 
    #data_epoch = mne.make_fixed_length_epochs(data_ch_dict_n, duration= interval, preload=True, id= csv_row)
    print('Fixed Length Epochs',csv_row,data_epoch, len(data_epoch), type(data_epoch) )
    #stronger_reject_criteria = dict(eeg=1000e-6)        #10microV
    #data_epoch.drop_bad(reject=stronger_reject_criteria)
    #print(data_epoch.drop_log)
    #print('Epochs return value',data_epoch.get_data(), np.shape(data_epoch.get_data()), type(data_epoch.get_data()) )
    
    data_labels = [None] * len(data_epoch)
    time_labels = [None] * len(data_epoch)
    #print('Number of total samples in 1 file',len(data_epoch) )
    for seg in range(0,len(data_epoch)):
        df_start = int(seg*time_window)
        df_end = int((seg+1)*time_window)
        #count_labels = df_subset.loc[df_start:df_end,'target'].sum()
        count_labels = df.loc[df_start:df_end,'target'].sum()
        if count_labels > time_window/2: #Update this threshold as needed TODO
            #print('SEIZURE DETECTED', csv_row, ':', count_labels,'seg', seg, 'start',df_start,'end',df_end)
            label = 1
        else:
            #print('NO SEIZURE DETECTED FOR', csv_row,':',count_labels,'seg', seg, 'start',df_start,'end',df_end)
            label = 0
        data_labels[seg] = label
        count_labels = 0
        time_labels[seg] = df.iloc[df_start:df_end,0]
    #print( np.shape(np.array(time_labels)),type(np.array(time_labels)) )   #RETURN
    #print('time_labels',time_labels)
    print('Label generated for',csv_row,':',data_epoch[0])
    
    #Generate time_labels
    
    # Non trimmed
    #print(df)
    '''
    t1 = df.time_s
    t2 = df.time_s

    plt.figure()
    plt.subplot(511)
    plt.plot(t1, df.iloc[:,1])
    plt.subplot(512)
    plt.plot(t1, df.iloc[:,2])
    plt.subplot(513)
    plt.plot(t1, df.iloc[:,3])
    plt.subplot(514)
    plt.plot(t1, df.iloc[:,4])
    #plt.plot(t1, df.TLB1)

    plt.subplot(515)
    plt.plot(t2, df.target)
    plt.show()#'''
    '''
    #print(df_subset)  # Trimmed
    t1 = df_subset.time_s
    t2 = df_subset.time_s

    plt.figure()
    plt.subplot(211)
    plt.plot(t1, df_subset.HRB2)
    #plt.plot(t1, df_subset.TLB1)

    plt.subplot(212)
    plt.plot(t2, df_subset.target)
    plt.show()#'''
    return data_epoch.get_data() , np.array(data_labels), np.array(time_labels), seizure_length

In [ ]:
#Pkl and unpkl all seizures for all patients
import pickle
with open('patient_id.pkl','wb') as pickle_file:
    pickle.dump(df,pickle_file)
    
with open('patient_id.pkl','rb') as pickle_file:
    pickle.load(pickle_file, same_df)

In [ ]:
cnt =0
for csv_row,data_raw in enumerate(reref_raws):
    #print(csv_row,'raw data', data_raw)
    print(np.shape(process_file(data_raw, csv_row,interval)[0]))
    #print('###################################################')
    #print(np.shape(process_file(data_raw, csv_row,interval)[1]))
    #print('###################################################')
    #print(np.shape(process_file(data_raw, csv_row,interval)[2]))
    #cnt +=1
    #if cnt >3 :
    #print(np.shape(process_file(data_raw, csv_row,interval)[0]), np.shape(process_file(data_raw, csv_row,interval)[1]))
    #    break
    break

In [ ]:
from torch.utils.data import Dataset, ConcatDataset
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor

class EpochsDataset(Dataset):
    """Class to expose an MNE Epochs object as PyTorch dataset.
    
    Parameters
    ----------
    epochs_data : np.ndarray
        The epochs data, shape (n_epochs, n_channels, n_times).
    epochs_labels : np.ndarray
        The epochs labels, shape (n_epochs,)
    epochs_time_labels : np.ndarray
        The time labels, shape (n_epochs,n_times)
    subj_num: None | int
        Subject number.
    rec_num: None | int
        Recording number.
    transform : callable | None
        The function to eventually apply to each epoch
        for preprocessing (e.g. scaling). Defaults to None.
    """
    def __init__(self, epochs_data, epochs_labels, epochs_time_labels,subj_num=None, rec_num=None,transform=None):
        assert len(epochs_data) == len(epochs_labels)
        self.epochs_data = epochs_data
        self.epochs_labels = epochs_labels
        self.epochs_time_labels = epochs_time_labels
        self.subj_num = subj_num
        self.rec_num = rec_num
        self.transform = transform

    def __len__(self):
        return len(self.epochs_labels)

    def __getitem__(self, idx):
        X, y = self.epochs_data[idx], self.epochs_labels[idx]
        time_label = self.epochs_time_labels[idx]
        #print('check getitem',X.shape,y.shape)
        if self.transform is not None:
            X = self.transform(X)
        X = torch.as_tensor(X[None, ...])
        return X, y, time_label

#transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
def scale(X):
    """Standard scaling of data along the last dimention.
    
    Parameters
    ----------
    X : array, shape (n_channels, n_times)
        The input signals.
        
    Returns
    -------
    X_t : array, shape (n_channels, n_times)
        The scaled signals.
    """
    #print('Input',X)
    X -= np.mean(X, axis=1, keepdims=True)
    #print('Mean',np.mean(X, axis=1, keepdims=True))
    #print('Std Dev',np.std(X, axis=1, keepdims=True))
    #print('Return', X / np.std(X, axis=1, keepdims=True))
    return X / np.std(X, axis=1, keepdims=True)

#TODO: ''######## IMPORTANT #########''torch.utils.data. needed? and if it is actually normalized 
transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))  
        ])

# To remove seizures less than 10s
all_datasets = []
cnt_discarded_seizures = 0
tmp_cnt = 0

for csv_row,data_raw in enumerate(reref_raws):
    r_data, r_label, r_time_label, len_seizure = process_file(data_raw, csv_row, interval)
    if len_seizure >= 10.1:
        valid_dataset = EpochsDataset(r_data, r_label, r_time_label, subj_num=data_raws[csv_row].info['subject_info']['id'], 
                              rec_num=data_raws[csv_row].info['subject_info']['rec_id'], transform=scale)
        tmp_cnt += 1
        all_datasets.append(valid_dataset) 
    else:
        cnt_discarded_seizures +=1
        print('The length of the seizure is',len_seizure,'s. Hence, discarded!')

print('Number of selected datasets are:',len(all_datasets),'. Number of seizures less than 10s:',cnt_discarded_seizures)
# Does not remove seizures less than 10s 
'''all_datasets = [EpochsDataset(*process_file(data_raw, csv_row, interval), subj_num=data_raws[csv_row].info['subject_info']['id'], 
                              rec_num=data_raws[csv_row].info['subject_info']['rec_id'], transform=scale) 
                for csv_row,data_raw in enumerate(reref_raws)]  ''' #replaced from data_raws

# Concatenate into a single dataset
dataset = ConcatDataset(all_datasets)

In [ ]:
print(all_datasets)

In [ ]:
print('data',all_datasets[0].epochs_data.shape)
print('label',all_datasets[0].epochs_labels.shape)
print('time_info',all_datasets[0].epochs_time_labels.shape)

#print('data',all_datasets[0].epochs_data)
print('label',all_datasets[0].epochs_labels)
print('time_info',all_datasets[0].epochs_time_labels)


In [ ]:
#TODO
#1. Test file split  - flag, divide
#2. Dataloader with timelabel info
#3. Randomsampler for selecting train-test in dataloader

In [ ]:
# Making training test validation split: 80-20
from sklearn.model_selection import LeavePGroupsOut  
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

def train_test_split(dataset):
    """Split dataset into train and test 
    
    Parameters
    ----------
    dataset : ConcatDataset
        The dataset to split.
        
    Returns
    -------
    ConcatDataset
        The training data and the testing data.
    """
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    print('Train and test size', train_size, test_size)
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    return train_dataset, test_dataset

torch.manual_seed(87)
np.random.seed(87)
train_ds, test_ds = train_test_split(dataset)
print('Number of examples in each set:')
print(f'Training: {len(train_ds)}')
print(f'Test: {len(test_ds)}')
print(train_ds)

In [ ]:
def reset_weights(m):
  """
    Resetting model weights to avoid weight leakage.
  """
  for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            #print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [ ]:
# CREATING NEURAL NETWORK  # 512Hz, 5sec
from torch import nn
import torch.nn.functional as F

class IEEGSeizureDetection(nn.Module):
    """
    Convolutional neural network 
    X_shape torch.Size([180, 1, 2, 2560]) Y_shape torch.Size([180])
    Parameters
    ----------
    n_channels : int
        Number of EEG channels.
    sfreq : float
        EEG sampling frequency. 512 Hz
    n_conv_chs : int
        Number of convolutional channels. Set to 2.
    time_conv_size_s : float
        Size of filters in temporal convolution layers, in seconds. 
        (512 samples at sfreq=512). e.g Use 0.5 for 256 samples
    max_pool_size_s : float
        Max pooling size, in seconds. 
    n_classes : int
        Number of classes.
    input_size_s : float
        Size of the input, in seconds.
    dropout : float
        Dropout rate before the output dense layer.
    """
    
    def __init__(self, n_channels, sfreq, n_conv_chs=15, time_conv_size_s=0.5,
                 max_pool_size_s=0.03125, n_classes=2, input_size_s=5,
                 dropout=0.25):
        super(IEEGSeizureDetection, self).__init__()

        time_conv_size = int(time_conv_size_s * sfreq) #256
        max_pool_size = int(max_pool_size_s * sfreq)   #16
        input_size = int(input_size_s * sfreq)         #2560
        pad_size = time_conv_size // 2                 #128
        self.n_channels = 4                   #2
        len_last_layer = self._len_last_layer(
            n_channels, input_size, max_pool_size, n_conv_chs) #300
        beta = 0.89
        print('Check parameters',time_conv_size,max_pool_size,input_size,pad_size,self.n_channels, len_last_layer)
        # 256 16 2560 128 2 300
        '''
        if n_channels > 1:
            self.spatial_conv = nn.Conv2d(1, n_channels, (n_channels, 1))

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, n_conv_chs, (1, time_conv_size), padding=(0, pad_size)),
            nn.BatchNorm2dd(n_conv_chs),
            nn.ReLU(),
            nn.MaxPool2d((1, max_pool_size)),
            nn.Conv2d(n_conv_chs, n_conv_chs, (1, time_conv_size),padding=(0, pad_size)),
            nn.BatchNorm2d(n_conv_chs),
            nn.ReLU(),
            nn.MaxPool2d((1, max_pool_size))
        )
        self.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(len_last_layer, n_classes)
        )'''
        self.conv1 = nn.Conv2d(1, 15, (n_channels,25) )
        self.batchnorm1 = nn.BatchNorm2d(15)
        self.maxpool = nn.MaxPool2d((1,n_channels))
        self.conv2 = nn.Conv2d(15, 15 , (1,11))
        self.batchnorm2 = nn.BatchNorm2d(15)
        self.conv3 = nn.Conv2d(15, 10 , (1,5))
        self.batchnorm3 = nn.BatchNorm2d(10)
        self.fc1 = nn.Linear(10*1*38, 2)
        self.dropout = nn.Dropout(0.2)

    @staticmethod
    def _len_last_layer(n_channels, input_size, max_pool_size, n_conv_chs):
        return n_channels * (input_size // (max_pool_size ** 2)) * n_conv_chs

    def forward(self, x):
        """Forward pass.
        
        Parameters
        ---------
        x: torch.Tensor
            Batch of EEG windows of shape (batch_size, n_channels, n_times).
        """
        '''
        if self.n_channels > 1:
            x = self.spatial_conv(x)
            x = x.transpose(1, 2)
        #[180, 1, 2, 2560]        
        x = self.feature_extractor(x)
        #([180, 15, 2, -- ])
        return self.fc(x.flatten(start_dim=1))
        #([180, 2])
        '''
        
        #print('input', x.size(),x)
        x = self.conv1(x)
        #print('conv1', x.size())
        # = self.batchnorm1(x)
        x = F.relu(x)
        #print('relu1', x.size())
        x = self.maxpool(x)
        x = self.dropout(x)
        #print('maxpool1', x.size())
        x = self.conv2(x)
        #print('conv2', x.size())
        x = self.batchnorm2(x)
        x = F.relu(x)
        #print('relu2', x.size())
        x = self.maxpool(x)
        #print('maxpool2', x.size())
        x = self.conv3(x)
        #print('conv3', x.size())
        x = self.batchnorm3(x)
        x = F.relu(x)
        #print('relu3', x.size())
        x = self.maxpool(x)
        #print('maxpool3', x.size())
        x = self.dropout(x)
        x = self.fc1(x.flatten(start_dim=1))
        #print('fc', x.size())

        return x


#input torch.Size([180, 1, 2, 2560])
#conv1 torch.Size([180, 15, 1, 2536])
#relu1 torch.Size([180, 15, 1, 2536])
#maxpool1 torch.Size([180, 15, 1, 634])
#conv2 torch.Size([180, 15, 1, 624])
#relu2 torch.Size([180, 15, 1, 624])
#maxpool2 torch.Size([180, 15, 1, 156])
#conv3 torch.Size([180, 10, 1, 152])
#relu3 torch.Size([180, 10, 1, 152])
#maxpool3 torch.Size([180, 10, 1, 38])
#fc torch.Size([180, 2])

In [ ]:
# CREATING NEURAL NETWORK  # 256Hz, 5sec
from torch import nn
import torch.nn.functional as F

class IEEGSeizureDetection_5sec(nn.Module):

    def __init__(self, n_channels, sfreq, n_conv_chs=15, time_conv_size_s=0.5,
                 max_pool_size_s=0.03125, n_classes=2, input_size_s=5,
                 dropout=0.25):
        super(IEEGSeizureDetection_5sec, self).__init__()

        time_conv_size = int(time_conv_size_s * sfreq) #256
        max_pool_size = int(max_pool_size_s * sfreq)   #16
        input_size = int(input_size_s * sfreq)         #2560
        pad_size = time_conv_size // 2                 #128
        self.n_channels = 4                   #2
        len_last_layer = self._len_last_layer(
            n_channels, input_size, max_pool_size, n_conv_chs) #300
        beta = 0.89
        print('Check parameters',time_conv_size,max_pool_size,input_size,pad_size,self.n_channels, len_last_layer)
        # 256 16 2560 128 2 300

        self.conv1 = nn.Conv2d(1, 15, (n_channels,25) )
        self.batchnorm1 = nn.BatchNorm2d(15)
        self.maxpool1 = nn.MaxPool2d((1,n_channels))
        self.dropout = nn.Dropout(0.2)
        self.conv2 = nn.Conv2d(15, 15 , (1,11))
        self.batchnorm2 = nn.BatchNorm2d(15)
        self.maxpool2 = nn.MaxPool2d((1,2))
        self.conv3 = nn.Conv2d(15, 10 , (1,5))
        self.batchnorm3 = nn.BatchNorm2d(10)
        self.maxpool3 = nn.MaxPool2d((1,2))
        self.conv4 = nn.Conv2d(10, 10 , (1,5))
        self.batchnorm4 = nn.BatchNorm2d(10)
        self.maxpool4 = nn.MaxPool2d((1,2))
        self.fc1 = nn.Linear(10*1*35, 2)


    @staticmethod
    def _len_last_layer(n_channels, input_size, max_pool_size, n_conv_chs):
        return n_channels * (input_size // (max_pool_size ** 2)) * n_conv_chs

    def forward(self, x):
        """Forward pass.
        
        Parameters
        ---------
        x: torch.Tensor
            Batch of EEG windows of shape (batch_size, n_channels, n_times).
        """
        #print('input', x.size())
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
        x = self.maxpool1(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        x = self.maxpool2(x)
        x = self.dropout(x)
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = F.relu(x)
        x = self.maxpool3(x)
        x = self.dropout(x)
        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = F.relu(x)
        x = self.maxpool4(x)
        x = self.dropout(x)
        x = self.fc1(x.flatten(start_dim=1))
        #x = torch.sigmoid(x)

        return x

In [ ]:
# CREATING NEURAL NETWORK with parameterization for freq = 256Hz and t= 1sec
from torch import nn
import torch.nn.functional as F

class IEEGSeizureDetection_1sec(nn.Module):
    """
    Convolutional neural network 
    X_shape torch.Size([180, 1, 4, 1280]) Y_shape torch.Size([180])
    Parameters
    ----------
    n_channels : int
        Number of EEG channels.
    sfreq : float
        EEG sampling frequency. 256 Hz
    n_conv_chs : int
        Number of convolutional channels. Set to 2.
    time_conv_size_s : float
        Size of filters in temporal convolution layers, in seconds. 
        (512 samples at sfreq=512). e.g Use 0.5 for 256 samples
    max_pool_size_s : float
        Max pooling size, in seconds. 
    n_classes : int
        Number of classes.
    input_size_s : float
        Size of the input, in seconds.
    dropout : float
        Dropout rate before the output dense layer.
    """
    
    def __init__(self, n_channels, sfreq, n_conv_chs=15, time_conv_size_s=0.5,
                 max_pool_size_s=0.03125, n_classes=2, input_size_s=5,
                 dropout=0.25):
        super(IEEGSeizureDetection_1sec, self).__init__()

        time_conv_size = int(time_conv_size_s * sfreq) #256
        max_pool_size = int(max_pool_size_s * sfreq)   #16
        input_size = int(input_size_s * sfreq)         #2560
        pad_size = time_conv_size // 2                 #128
        self.n_channels = 4                   #2
        len_last_layer = self._len_last_layer(
            n_channels, input_size, max_pool_size, n_conv_chs) #300
        beta = 0.89
        print('Check parameters',time_conv_size,max_pool_size,input_size,pad_size,self.n_channels, len_last_layer)
        # 256 16 2560 128 2 300
        '''
        if n_channels > 1:
            self.spatial_conv = nn.Conv2d(1, n_channels, (n_channels, 1))

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, n_conv_chs, (1, time_conv_size), padding=(0, pad_size)),
            nn.BatchNorm2dd(n_conv_chs),
            nn.ReLU(),
            nn.MaxPool2d((1, max_pool_size)),
            nn.Conv2d(n_conv_chs, n_conv_chs, (1, time_conv_size),padding=(0, pad_size)),
            nn.BatchNorm2d(n_conv_chs),
            nn.ReLU(),
            nn.MaxPool2d((1, max_pool_size))
        )
        self.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(len_last_layer, n_classes)
        )'''
        self.conv1 = nn.Conv2d(1, 15, (n_channels,25) )
        self.batchnorm1 = nn.BatchNorm2d(15)
        self.maxpool = nn.MaxPool2d((1,4))
        self.dropout = nn.Dropout(0.2)
        self.conv2 = nn.Conv2d(15, 15 , (1,11))
        self.batchnorm2 = nn.BatchNorm2d(15)
        self.conv3 = nn.Conv2d(15, 10 , (1,5))
        self.batchnorm3 = nn.BatchNorm2d(10)
        self.fc1 = nn.Linear(10*1*8, 8)
        self.fc2 = nn.Linear(8, 2)  # TODO : output 1 or 2
        #self.fc1 = nn.Linear(10*1*154, 1000)
        #self.fc2 = nn.Linear(1000, 300)
        #self.fc3 = nn.Linear(300, 2)

    @staticmethod
    def _len_last_layer(n_channels, input_size, max_pool_size, n_conv_chs):
        return n_channels * (input_size // (max_pool_size ** 2)) * n_conv_chs

    def forward(self, x):
        """Forward pass.
        
        Parameters
        ---------
        x: torch.Tensor
            Batch of EEG windows of shape (batch_size, n_channels, n_times).
        """
        '''
        if self.n_channels > 1:
            x = self.spatial_conv(x)
            x = x.transpose(1, 2)
        #[180, 1, 2, 2560]        
        x = self.feature_extractor(x)
        #([180, 15, 2, -- ])
        return self.fc(x.flatten(start_dim=1))
        #([180, 2])
        '''
        
        #print('input', x.size())
        x = self.conv1(x)
        #print('conv1', x.size())
        x = self.batchnorm1(x)
        x = F.relu(x)
        #print('relu1', x.size())
        x = self.maxpool(x)
        x = self.dropout(x)
        #print('maxpool1', x.size())
        x = self.conv2(x)
        #print('conv2', x.size())
        x = self.batchnorm2(x)
        x = F.relu(x)
        #print('relu2', x.size())
        x = self.maxpool(x)
        x = self.dropout(x)
        #print('maxpool2', x.size())
        x = self.conv3(x)
        #print('conv3', x.size())
        x = self.batchnorm3(x)
        x = F.relu(x)
        #print('relu3', x.size())
        #x = self.maxpool(x)
        #print('maxpool3', x.size())
        x = self.dropout(x)
        x = self.fc1(x.flatten(start_dim=1))
        x = self.fc2(x)
        #x = self.fc3(x)
        #print('fc', x.size())
        #x = torch.sigmoid(x)

        return x


#input torch.Size([180, 1, 2, 2560])
#conv1 torch.Size([180, 15, 1, 2536])
#relu1 torch.Size([180, 15, 1, 2536])
#maxpool1 torch.Size([180, 15, 1, 634])
#conv2 torch.Size([180, 15, 1, 624])
#relu2 torch.Size([180, 15, 1, 624])
#maxpool2 torch.Size([180, 15, 1, 156])
#conv3 torch.Size([180, 10, 1, 152])
#relu3 torch.Size([180, 10, 1, 152])
#maxpool3 torch.Size([180, 10, 1, 38])
#fc torch.Size([180, 2])

In [ ]:
#Conv1D
#conv1D
# CREATING NEURAL NETWORK with parameterization for freq = 256Hz and t= 2sec : based on Farrokh's
from torch import nn
import torch.nn.functional as F

class IEEGSeizureDetection_2sec(nn.Module):
    
    def __init__(self, trial, params, n_channels, sfreq, n_conv_chs=15, time_conv_size_s=0.5,
                 max_pool_size_s=0.03125, n_classes=2, input_size_s=5, dropout=0.25):
        super(IEEGSeizureDetection_2sec, self).__init__()

        time_conv_size = int(time_conv_size_s * sfreq) #256
        max_pool_size = int(max_pool_size_s * sfreq)   #16
        input_size = int(input_size_s * sfreq)         #2560
        pad_size = time_conv_size // 2                 #128
        self.n_channels = 4                            #2
        len_last_layer = self._len_last_layer(n_channels, input_size, max_pool_size, n_conv_chs) #300
        beta = 0.89
        print('Check parameters',time_conv_size,max_pool_size,input_size,pad_size,self.n_channels, len_last_layer)

        #Quantized Aware Training
        #8-bit signed integer with a per-tensor floating-point scale factor (ONNX standard)
        '''
        weight_bit_width = 4
        act_bit_width = 4
        #Layer1
        self.quant_identity = QuantIdentity(return_quant_tensor=True)
        self.conv1 = qnn.QuantConv1d(in_channels=4, out_channels= 20,kernel_size= 17,
                                     bias=True,
                                     #weight_bit_width=weight_bit_width,
                                     #input_quant=Int8ActPerTensorFloat, 
                                     #weight_quant=Int8ActPerTensorFloat,
                                     bias_quant=Int8Bias,
                                     #output_quant=Int8ActPerTensorFloat,
                                     return_quant_tensor=True)
        #print(self.conv1)
        self.batchnorm1 = nn.BatchNorm1d(20)
        self.maxpool1 = nn.MaxPool1d(kernel_size=4)
        self.dropout = QuantDropout(0.2)
        self.relu1 = QuantReLU(return_quant_tensor=True)
        
        #Layer2
        self.conv2 = qnn.QuantConv1d(in_channels=20, out_channels=10 ,kernel_size= 5,
                                     bias=True,
                                     #weight_bit_width=weight_bit_width,
                                     #input_quant=Int8ActPerTensorFloat, 
                                     bias_quant=Int8Bias,
                                     #output_quant=Int8ActPerTensorFloat,
                                     return_quant_tensor=True)
        #print(self.conv2)
        self.batchnorm2 = nn.BatchNorm1d(10)
        self.maxpool2 = nn.MaxPool1d(kernel_size=4)
        self.relu2 = QuantReLU(return_quant_tensor=True)
        
        #Layer3
        self.conv3 = qnn.QuantConv1d(in_channels=10, out_channels=10 , kernel_size=5,
                                     bias=True,
                                     #weight_bit_width=weight_bit_width,
                                     #input_quant=Int8ActPerTensorFloat, 
                                     bias_quant=Int8Bias,
                                     #output_quant=Int8ActPerTensorFloat,
                                     return_quant_tensor=True)
        #print(self.conv3)
        self.batchnorm3 = nn.BatchNorm1d(10)
        self.maxpool3 = nn.MaxPool1d(kernel_size=2)
        self.relu3 = QuantReLU(return_quant_tensor=True)
        
        #Layer4
        self.conv4 = qnn.QuantConv1d(in_channels=10, out_channels=10 , kernel_size=5,
                                     bias=True,
                                     #weight_bit_width=weight_bit_width,
                                     #input_quant=Int8ActPerTensorFloat, 
                                     bias_quant=Int8Bias,
                                     #output_quant=Int8ActPerTensorFloat,
                                     return_quant_tensor=True)
        #print(self.conv4)
        self.batchnorm4 = nn.BatchNorm1d(10)
        self.relu4 = QuantReLU(return_quant_tensor=True)
        
        #Layer5
        self.fc1 = qnn.QuantLinear(in_features=10*1*9, out_features= 5, 
                                   bias=True,   
                                   #weight_bit_width=weight_bit_width,
                                   #output_quant=Int8ActPerTensorFloat,
                                   bias_quant=Int8Bias,
                                   return_quant_tensor=True)
        #print(self.fc1)
        self.relu5 = QuantReLU(return_quant_tensor=True)
        
        #Layer6
        self.fc2 = qnn.QuantLinear(in_features=5, out_features=5, 
                                   bias=True,  
                                   #weight_bit_width=weight_bit_width,
                                   #output_quant=Int8ActPerTensorFloat,
                                   bias_quant=Int8Bias,
                                   return_quant_tensor=True)
        #print(self.fc2)
        self.relu6 = QuantReLU(return_quant_tensor=True)
        
        #Layer7
        self.fc3 = qnn.QuantLinear(in_features=5, 
                                   #out_features=1, 
                                   #input_quant=Int8ActPerTensorFloat, 
                                   out_features=2,
                                   bias=True, 
                                   #weight_bit_width=weight_bit_width,
                                   #output_quant=Int8ActPerTensorFloat,
                                   bias_quant=Int8Bias,
                                   return_quant_tensor=True)
        #print(self.fc3)
        #self.sigmoid = QuantSigmoid(return_quant_tensor=True)
        #self.sigmoid = nn.Sigmoid()
        #'''
        # Non Quantized
        '''
        self.conv1 = nn.Conv1d(in_channels=4, out_channels= 20,kernel_size= 17 )
        self.batchnorm1 = nn.BatchNorm1d(20)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=n_channels)
        self.dropout = nn.Dropout(0.2)
        self.conv2 = nn.Conv1d(in_channels=20, out_channels=10 ,kernel_size= 5)
        self.batchnorm2 = nn.BatchNorm1d(10)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=n_channels)
        self.conv3 = nn.Conv1d(in_channels=10, out_channels=10 , kernel_size=5)
        self.batchnorm3 = nn.BatchNorm1d(10)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool1d(kernel_size=2)
        self.conv4 = nn.Conv1d(in_channels=10, out_channels=10 , kernel_size=5)
        self.batchnorm4 = nn.BatchNorm1d(10)
        self.relu4 = nn.ReLU()
        self.fc1 = nn.Linear(in_features=10*1*9, out_features= 5, bias=True)
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=5, out_features=5, bias=True)
        self.relu6 = nn.ReLU()
        self.fc3 = nn.Linear(in_features=5, out_features=2, bias=True)
        #self.sigmoid = nn.Sigmoid()#'''

        #Hyperparameterized
        print('Chosen Hyperparameters',params, type(params), len(params))
        #print('n_filters_1' in params)
        #print('check val',params['n_filters_1'] )
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=params['n_filters_1'] ,kernel_size= 17 )
        self.batchnorm1 = nn.BatchNorm1d(params['n_filters_1'])
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=n_channels)
        self.dropout = nn.Dropout(0.2)
        self.conv2 = nn.Conv1d(in_channels=params['n_filters_1'], out_channels=params['n_filters_2'] ,kernel_size= 5)
        self.batchnorm2 = nn.BatchNorm1d(params['n_filters_2'])
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=n_channels)
        self.conv3 = nn.Conv1d(in_channels=params['n_filters_2'], out_channels=params['n_filters_3'], kernel_size= 5)
        self.batchnorm3 = nn.BatchNorm1d(params['n_filters_3'])
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool1d(kernel_size=2)
        self.conv4 = nn.Conv1d(in_channels=params['n_filters_3'], out_channels=params['n_filters_4'] , kernel_size= 5)
        self.batchnorm4 = nn.BatchNorm1d(params['n_filters_4'])
        self.relu4 = nn.ReLU()
        self.fc1 = nn.Linear(in_features=params['n_filters_4']*1*9, out_features=params['n_output_fc1'], bias=True)
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=params['n_output_fc1'], out_features= params['n_output_fc2'], bias=True)
        self.relu6 = nn.ReLU()
        self.fc3 = nn.Linear(in_features=params['n_output_fc2'], out_features=2, bias=True)  # TODO: Output 1 or 2

    @staticmethod
    def _len_last_layer(n_channels, input_size, max_pool_size, n_conv_chs):
        return n_channels * (input_size // (max_pool_size ** 2)) * n_conv_chs

    def forward(self, x):
        """Forward pass.
        
        Parameters
        ---------
        x: torch.Tensor
            Batch of EEG windows of shape (batch_size, n_channels, n_times).
        """
        #print('in', x.shape)
        #x = self.quant_identity(x)  # UNCOMMENT FOR QUANTIZATION 
        #print(f"Input has QuantTensor:\n {x.is_valid} \n")
        #print('conv1', x.shape)
        x = self.conv1(x)
        #print(f"CONV1:\n {x} \n")
        #print(f"CONV1 has QuantTensor:\n {x.is_valid} \n")
        #print('bn', x.shape)
        x = self.batchnorm1(x)
        #print(f"BN1 has QuantTensor:\n {x.is_valid} \n")
        #print('relu', x.shape)
        x = self.relu1(x)
        #print(f"R1 has QuantTensor:\n {x.is_valid} \n")
        #print('maxpool', x.shape)
        x = self.maxpool1(x)
        #print(f"MP1 has QuantTensor:\n {x.is_valid} \n")
        #print('dropout', x.shape)
        x = self.dropout(x)
        #print(f"DR1 has QuantTensor:\n {x.is_valid} \n")
        #print('conv2 shape', x.shape)
        x = self.conv2(x)
        #print(f"CONV2:\n {x} \n")
        #print(f"CONV2 has QuantTensor:\n {x.is_valid} \n")
        x = self.batchnorm2(x)
        #print(f"BN2 has QuantTensor:\n {x.is_valid} \n")
        x = self.relu2(x)
        #print(f"R2 has QuantTensor:\n {x.is_valid} \n")
        x = self.maxpool2(x)
        #print(f"MP2 has QuantTensor:\n {x.is_valid} \n")
        x = self.dropout(x)
        #print(f"DR2 has QuantTensor:\n {x.is_valid} \n")
        x = self.conv3(x)
        #print(f"CONV3:\n {x} \n")
        #print(f"CONV3 has QuantTensor:\n {x.is_valid} \n")
        x = self.batchnorm3(x)
        #print(f"BN3 has QuantTensor:\n {x.is_valid} \n")
        x = self.relu3(x)
        #print(f"R3 has QuantTensor:\n {x.is_valid} \n")
        x = self.maxpool3(x)
        #print(f"MP3 has QuantTensor:\n {x.is_valid} \n")
        x = self.dropout(x)
        #print(f"DR3 has QuantTensor:\n {x.is_valid} \n")
        x = self.conv4(x)
        #print(f"CONV4:\n {x} \n")
        #print(f"CONV4 has QuantTensor:\n {x.is_valid} \n")
        x = self.batchnorm4(x)
        #print(f"BN4 has QuantTensor:\n {x.is_valid} \n")
        x = self.relu4(x)
        #print(f"R4 has QuantTensor:\n {x.is_valid} \n")
        x = self.dropout(x)
        x = self.relu5(self.fc1(x.flatten(start_dim=1)))
        #print(f"FC1:\n {x} \n")
        #print(f"FC1 has QuantTensor:\n {x.is_valid} \n")
        x = self.relu6(self.fc2(x))
        #print(f"FC2:\n {x} \n")
        #print(f"FC2 has QuantTensor:\n {x.is_valid} \n")
        #x = self.sigmoid(self.fc3(x))
        x = self.fc3(x)
        #print(f"FC3:\n {x} \n")
        #print(f"FC3 has QuantTensor:\n {x.is_valid} \n")

        return x

In [ ]:
# CREATING NEURAL NETWORK with parameterization for freq = 256Hz and t= 2sec : based on Farrokh's
from torch import nn
import torch.nn.functional as F

class IEEGSeizureDetection_2sec(nn.Module):
    """
    Convolutional neural network 
    X_shape torch.Size([180, 1, 4, 1280]) Y_shape torch.Size([180])
    Parameters
    ----------
    n_channels : int
        Number of EEG channels.
    sfreq : float
        EEG sampling frequency. 256 Hz
    n_conv_chs : int
        Number of convolutional channels. Set to 2.
    time_conv_size_s : float
        Size of filters in temporal convolution layers, in seconds. 
        (512 samples at sfreq=512). e.g Use 0.5 for 256 samples
    max_pool_size_s : float
        Max pooling size, in seconds. 
    n_classes : int
        Number of classes.
    input_size_s : float
        Size of the input, in seconds.
    dropout : float
        Dropout rate before the output dense layer.
    """
    
    def __init__(self, n_channels, sfreq, n_conv_chs=15, time_conv_size_s=0.5,
                 max_pool_size_s=0.03125, n_classes=2, input_size_s=5, dropout=0.25):
        super(IEEGSeizureDetection_2sec, self).__init__()

        time_conv_size = int(time_conv_size_s * sfreq) #256
        max_pool_size = int(max_pool_size_s * sfreq)   #16
        input_size = int(input_size_s * sfreq)         #2560
        pad_size = time_conv_size // 2                 #128
        self.n_channels = 4                            #2
        len_last_layer = self._len_last_layer(n_channels, input_size, max_pool_size, n_conv_chs) #300
        beta = 0.89
        print('Check parameters',time_conv_size,max_pool_size,input_size,pad_size,self.n_channels, len_last_layer)
        # 256 16 2560 128 2 300
        '''
        if n_channels > 1:
            self.spatial_conv = nn.Conv2d(1, n_channels, (n_channels, 1))

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, n_conv_chs, (1, time_conv_size), padding=(0, pad_size)),
            nn.BatchNorm2dd(n_conv_chs),
            nn.ReLU(),
            nn.MaxPool2d((1, max_pool_size)),
            nn.Conv2d(n_conv_chs, n_conv_chs, (1, time_conv_size),padding=(0, pad_size)),
            nn.BatchNorm2d(n_conv_chs),
            nn.ReLU(),
            nn.MaxPool2d((1, max_pool_size))
        )
        self.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(len_last_layer, n_classes)
        )'''

        self.conv1 = nn.Conv2d(in_channels=1, out_channels= 20,kernel_size= (n_channels,17) )
        self.batchnorm1 = nn.BatchNorm2d(20)
        self.maxpool1 = nn.MaxPool2d(kernel_size=(1,n_channels))
        self.dropout = nn.Dropout(0.2)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=10 ,kernel_size= (1,5))
        self.batchnorm2 = nn.BatchNorm2d(10)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(1,n_channels))
        self.conv3 = nn.Conv2d(in_channels=10, out_channels=10 , kernel_size=(1,5))
        self.batchnorm3 = nn.BatchNorm2d(10)
        self.maxpool3 = nn.MaxPool2d(kernel_size=(1,2))
        self.conv4 = nn.Conv2d(in_channels=10, out_channels=10 , kernel_size=(1,5))
        self.batchnorm4 = nn.BatchNorm2d(10)
        self.fc1 = nn.Linear(in_features=10*1*9, out_features= 5, bias=True)
        self.fc2 = nn.Linear(in_features=5, out_features=5, bias=True)
        self.fc3 = nn.Linear(in_features=5, out_features=2, bias=True) 

    @staticmethod
    def _len_last_layer(n_channels, input_size, max_pool_size, n_conv_chs):
        return n_channels * (input_size // (max_pool_size ** 2)) * n_conv_chs

    def forward(self, x):
        """Forward pass.
        
        Parameters
        ---------
        x: torch.Tensor
            Batch of EEG windows of shape (batch_size, n_channels, n_times).
        """
      
        #print('input', x.size())
        x = self.conv1(x)
        #print('conv1', x.size())
        x = self.batchnorm1(x)
        x = F.relu(x)
        #print('relu1', x.size())
        x = self.maxpool1(x)
        x = self.dropout(x)
        #print('maxpool1', x.size())
        x = self.conv2(x)
        #print('conv2', x.size())
        x = self.batchnorm2(x)
        x = F.relu(x)
        #print('relu2', x.size())
        x = self.maxpool2(x)
        x = self.dropout(x)
        #print('maxpool2', x.size())
        x = self.conv3(x)
        #print('conv3', x.size())
        x = self.batchnorm3(x)
        x = F.relu(x)
        #print('relu3', x.size())
        x = self.maxpool3(x)
        #print('maxpool3', x.size())
        x = self.dropout(x)
        x = self.conv4(x)
        #print('conv4', x.size())
        x = self.batchnorm4(x)
        x = F.relu(x)
        #print('relu4', x.size())
        x = self.dropout(x)
        x = F.relu(self.fc1(x.flatten(start_dim=1)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        #x = torch.sigmoid(x)
        #print('fc', x.size())

        return x

In [ ]:
# CREATING NEURAL NETWORK with parameterization for freq = 256Hz and t= 2sec : based on Farrokh's + HPO
from torch import nn
import torch.nn.functional as F

class IEEGSeizureDetection_2sec(nn.Module):
    """
    Convolutional neural network 
    X_shape torch.Size([180, 1, 4, 1280]) Y_shape torch.Size([180])
    Parameters
    ----------
    n_channels : int
        Number of EEG channels.
    sfreq : float
        EEG sampling frequency. 256 Hz
    n_conv_chs : int
        Number of convolutional channels. Set to 2.
    time_conv_size_s : float
        Size of filters in temporal convolution layers, in seconds. 
        (512 samples at sfreq=512). e.g Use 0.5 for 256 samples
    max_pool_size_s : float
        Max pooling size, in seconds. 
    n_classes : int
        Number of classes.
    input_size_s : float
        Size of the input, in seconds.
    dropout : float
        Dropout rate before the output dense layer.
    """
    
    def __init__(self, trial, params, n_channels, sfreq, n_conv_chs=15, time_conv_size_s=0.5,
                 max_pool_size_s=0.03125, n_classes=2, input_size_s=5, dropout=0.25):
        super(IEEGSeizureDetection_2sec, self).__init__()

        time_conv_size = int(time_conv_size_s * sfreq) #256
        max_pool_size = int(max_pool_size_s * sfreq)   #16
        input_size = int(input_size_s * sfreq)         #2560
        pad_size = time_conv_size // 2                 #128
        self.n_channels = 4                            #2
        len_last_layer = self._len_last_layer(n_channels, input_size, max_pool_size, n_conv_chs) #300
        beta = 0.89
        #print('Check parameters',time_conv_size,max_pool_size,input_size,pad_size,self.n_channels, len_last_layer)
        # 256 16 2560 128 2 300
        '''
        if n_channels > 1:
            self.spatial_conv = nn.Conv2d(1, n_channels, (n_channels, 1))

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, n_conv_chs, (1, time_conv_size), padding=(0, pad_size)),
            nn.BatchNorm2dd(n_conv_chs),
            nn.ReLU(),
            nn.MaxPool2d((1, max_pool_size)),
            nn.Conv2d(n_conv_chs, n_conv_chs, (1, time_conv_size),padding=(0, pad_size)),
            nn.BatchNorm2d(n_conv_chs),
            nn.ReLU(),
            nn.MaxPool2d((1, max_pool_size))
        )
        self.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(len_last_layer, n_classes)
        )'''
        
        #self.params = params
        print('Chosen Hyperparameters',params, type(params), len(params))
        #print('n_filters_1' in params)
        #print('check val',params['n_filters_1'] )
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=params['n_filters_1'] ,kernel_size= (n_channels,17) )
        self.batchnorm1 = nn.BatchNorm2d(params['n_filters_1'])
        self.maxpool1 = nn.MaxPool2d(kernel_size=(1,n_channels))
        self.dropout = nn.Dropout(0.2)
        self.conv2 = nn.Conv2d(in_channels=params['n_filters_1'], out_channels=params['n_filters_2'] ,kernel_size= (1,5))
        self.batchnorm2 = nn.BatchNorm2d(params['n_filters_2'])
        self.maxpool2 = nn.MaxPool2d(kernel_size=(1,n_channels))
        self.conv3 = nn.Conv2d(in_channels=params['n_filters_2'], out_channels=params['n_filters_3'], kernel_size=(1,5))
        self.batchnorm3 = nn.BatchNorm2d(params['n_filters_3'])
        self.maxpool3 = nn.MaxPool2d(kernel_size=(1,2))
        self.conv4 = nn.Conv2d(in_channels=params['n_filters_3'], out_channels=params['n_filters_4'] , kernel_size=(1,5))
        self.batchnorm4 = nn.BatchNorm2d(params['n_filters_4'])
        self.fc1 = nn.Linear(in_features=params['n_filters_4']*1*9, out_features=params['n_output_fc1'], bias=True)
        self.fc2 = nn.Linear(in_features=params['n_output_fc1'], out_features= params['n_output_fc2'], bias=True)
        self.fc3 = nn.Linear(in_features=params['n_output_fc2'], out_features=2, bias=True)  # TODO: Output 1 or 2

    @staticmethod
    def _len_last_layer(n_channels, input_size, max_pool_size, n_conv_chs):
        return n_channels * (input_size // (max_pool_size ** 2)) * n_conv_chs

    def forward(self, x):
        """Forward pass.
        
        Parameters
        ---------
        x: torch.Tensor
            Batch of EEG windows of shape (batch_size, n_channels, n_times).
        """
      
        #print('input', x.size())
        x = self.conv1(x)
        #print('conv1', x.size())
        x = self.batchnorm1(x)
        x = F.relu(x)
        #print('relu1', x.size())
        x = self.maxpool1(x)
        x = self.dropout(x)
        #print('maxpool1', x.size())
        x = self.conv2(x)
        #print('conv2', x.size())
        x = self.batchnorm2(x)
        x = F.relu(x)
        #print('relu2', x.size())
        x = self.maxpool2(x)
        x = self.dropout(x)
        #print('maxpool2', x.size())
        x = self.conv3(x)
        #print('conv3', x.size())
        x = self.batchnorm3(x)
        x = F.relu(x)
        #print('relu3', x.size())
        x = self.maxpool3(x)
        #print('maxpool3', x.size())
        x = self.dropout(x)
        x = self.conv4(x)
        #print('conv4', x.size())
        x = self.batchnorm4(x)
        x = F.relu(x)
        #print('relu4', x.size())
        x = self.dropout(x)
        x = F.relu(self.fc1(x.flatten(start_dim=1)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        #x = torch.sigmoid(x)
        #print('fc', x.size())

        return x

In [ ]:
from torchsummary import summary
device = 'cuda'
#sfreq = data_raws[0].info['sfreq']  # Sampling frequency
sfreq = 256.0 
#n_channels = data_raws[0].info['nchan']  # Number of channels
n_channels = 4   #TODO : Get from function correct value
print(sfreq, n_channels)
model= IEEGSeizureDetection_2sec(n_channels, sfreq, n_classes=2)
model.apply(reset_weights)
print(f'Using device \'{device}\'.')
model = model.to(device)
print(model)

In [ ]:
def calculate_detection_delay_for_eval(df):
    #print(df)
    if ((df['True_Label'] == 0).all() and (df['Predicted_Label'] == 0).all()):
        print('STATUS : No seizure labeled or detected')
        diff = 0
        s_d = 0
    elif ((df['True_Label'] == 0).all() and not((df['Predicted_Label'] == 0).all())):
        print('STATUS : False detection happened')
        diff = 0
        s_d = 0
    elif (not((df['True_Label'] == 0).all()) and (df['Predicted_Label'] == 0).all()):
        print('STATUS : Detection Missed')
        diff = 0
        s_d = 0
    elif (not((df['True_Label'] == 0).all()) and not((df['Predicted_Label'] == 0).all())):
        #t_pred = df.index[df['Predicted_Label'] == 1].tolist()
        #t_true = df.index[df['True_Label'] == 1].tolist()
        t_true = np.where(df[ 'True_Label'] == 1)
        t_pred = np.where(df[ 'Predicted_Label'] == 1)
        #print('Indices', t_pred, t_true)
        #print('Indices', t_pred[0][0], t_true[0][0])
        if t_pred[0][0] <= t_true[0][0]: #False pos
            print('STATUS : False detection happened, but there is a seizure also')
        #print('Prediction time', df.iloc[t_pred[0][0],6], 'True time',df.iloc[t_true[0][0],6] )
        diff = (df.iloc[t_pred[0][0],6] - df.iloc[t_true[0][0],6])/1000.0 
        print('STATUS : Detection delay of',diff,'secs')
        s_d = 1
    return diff, s_d

In [ ]:

def plot_confusion_matrix(conf_mat):
    classes_mapping = {0: 'non-ictal', 1: 'ictal'} 
    ticks = list(classes_mapping.keys())
    tick_labels = classes_mapping.values()

    fig, ax = plt.subplots(figsize=(6, 6))
    im = ax.imshow(conf_mat, cmap='Reds')
    ax.set_yticks(ticks)
    ax.set_yticklabels(tick_labels)
    ax.set_xticks(ticks)
    ax.set_xticklabels(tick_labels)
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')
    ax.set_title('Confusion matrix')

    for i in range(len(ticks)):
        for j in range(len(ticks)):
            text = ax.text(
                j, i, conf_mat[i, j], ha='center', va='center', color='k')

    fig.colorbar(im, ax=ax, fraction=0.05, label='# examples')
    fig.tight_layout()
    #fig.savefig('./save_plot/confusion_matrix_'+i+'.png')
    return fig, ax

In [ ]:
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score
from sklearn.metrics import average_precision_score, f1_score,  auc
from sklearn.metrics import precision_recall_curve, confusion_matrix

def analog_to_spike(batch_x, batch_y):
    #print('shape for analog to spike',np.shape(batch_x), np.shape(batch_x)[0])
    b_s = np.shape(batch_x)[0]
    c_s = np.shape(batch_x)[2]
    t_s = np.shape(batch_x)[3]

    batch_x = spikegen.delta(batch_x, threshold=0.5, off_spike=False)
    #print('shape for loop',np.shape(batch_x), len(batch_x),b_s,c_s,t_s, 'shape of y',np.shape(batch_y))
    #print('Check the value to be plotted', batch_x[0,0,0,:], len(batch_x[0,0,0,:])
    '''
    for b in range(0,5):
        for c in range(0,c_s):    
            # Create fig, ax
            fig = plt.figure(facecolor="w", figsize=(8, 4))
            ax = fig.add_subplot(111)

            # Raster plot of delta converted data
            splt.raster(batch_x[b,0,c,:], ax)

            plt.title("Input Neuron"+ "for batchsample_"+ str(b) +"_for channel_"+ str(c) + "_with_label:" +str(batch_y[b]))
            plt.xlabel("Time step")
            plt.yticks([])
            plt.xlim(0,t_s )
            fig.canvas.draw()
    '''
    return batch_x
            

def _do_train(model, loader, optimizer, criterion, device, metric):
    # training loop
    model.train()
    
    train_loss = np.zeros(len(loader))
    y_pred_all, y_true_all = list(), list()
    for idx_batch, (batch_x, batch_y, batch_t) in enumerate(loader):
        optimizer.zero_grad()
        batch_x = batch_x.to(device=device, dtype=torch.float32)   #([180, 1, 2, 2560])
        batch_x = torch.squeeze(batch_x,1)
        batch_y = batch_y.to(device=device, dtype=torch.int64)
        output = model(batch_x)
        #output = torch.squeeze(output,1)(
        #batch_y = batch_y.unsqueeze(1).float()
        #print('actual class',batch_y, batch_y.shape)
        #print('predicted class', torch.argmax(output, axis=1).cpu().numpy())
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        
        y_pred_all.append(torch.argmax(output, axis=1).cpu().numpy())
        y_true_all.append(batch_y.cpu().numpy())

        train_loss[idx_batch] = loss.item()
        
    y_pred = np.concatenate(y_pred_all)
    y_true = np.concatenate(y_true_all)
    perf = metric(y_true, y_pred)
    acc = np.mean(y_true == y_pred)
    #print('TRAIN: y_pred',y_pred,'y_true',y_true )
    return np.mean(train_loss), perf   
        
'''
def _validate(model, loader, criterion, device, metric):
    # validation loop
    model.eval()
    
    val_loss = np.zeros(len(loader))
    y_pred_all, y_true_all = list(), list()
    with torch.no_grad():
        for idx_batch, (batch_x, batch_y, batch_t) in enumerate(loader):
            batch_x = batch_x.to(device=device, dtype=torch.float32)
            batch_y = batch_y.to(device=device, dtype=torch.int64)
            output = model.forward(batch_x)
            #output = torch.squeeze(output,1)
            #print('predicted class', torch.argmax(output, axis=1).cpu().numpy())
            #print('actual class', batch_y)
            loss = criterion(output, batch_y)
            val_loss[idx_batch] = loss.item()
            
            y_pred_all.append(torch.argmax(output, axis=1).cpu().numpy())
            y_true_all.append(batch_y.cpu().numpy())
            
    y_pred = np.concatenate(y_pred_all)
    y_true = np.concatenate(y_true_all)
    perf = metric(y_true, y_pred)
    #print('VALID: y_pred',y_pred,'y_true',y_true)    
    return np.mean(val_loss), perf'''

def _validate(model, loader, criterion, device, metric):
    # validation loop
    model.eval()
    
    average_precision = 0
    f1_score_eval = 0
    val_loss = np.zeros(len(loader))
    y_pred_all, y_true_all, y_score_all = list(), list(), list()
    
    with torch.no_grad():
        for idx_batch, (batch_x, batch_y, batch_t) in enumerate(loader):
            batch_x = batch_x.to(device=device, dtype=torch.float32)
            batch_x = torch.squeeze(batch_x,1)
            batch_y = batch_y.to(device=device, dtype=torch.int64)
            batch_t = batch_t
            output = model.forward(batch_x)
            #output = torch.squeeze(output,1)
            #print('predicted class', torch.argmax(output, axis=1).cpu().numpy())
            #print('actual class', batch_y)
            loss = criterion(output, batch_y)
            val_loss[idx_batch] = loss.item()
            
            pred_probab = nn.Softmax(dim=1)(output)  # Move this TODO
            probs = pred_probab[:, 1]
            
            y_pred_all.append(torch.argmax(output, axis=1).cpu().numpy())
            y_true_all.append(batch_y.cpu().numpy())
            y_score_all.append(probs.detach().cpu().numpy())
            #print('Debug Average Precsion','pred_probab',pred_probab,'probs',probs, 'y_score_all' ,y_score_all)
            
    y_pred = np.concatenate(y_pred_all)
    y_true = np.concatenate(y_true_all)
    y_score = np.concatenate(y_score_all)
    
    perf = metric(y_true, y_pred)
    f1_score_eval = f1_score(y_true, y_pred, average='binary')
    average_precision = average_precision_score(y_true, y_score)
    
    #print('VALID: No Mean',val_loss ,  np.mean(val_loss), perf, average_precision, f1_score_eval )  
    # return loss, accuracy, AP, F1 score, delay
    return np.mean(val_loss), perf, average_precision , f1_score_eval

def _validate_1epoch(model, loader, criterion, device, metric):
    # validation loop
    model.eval()
    
    average_precision = 0
    f1_score_eval = 0
    false_positives = 0
    detection_delay = 0
    val_loss = np.zeros(len(loader))
    y_pred_all, y_true_all, y_score_all = list(), list(), list()
    
    with torch.no_grad():
        for idx_batch, (batch_x, batch_y, batch_t) in enumerate(loader):
            batch_x = batch_x.to(device=device, dtype=torch.float32)
            batch_x = torch.squeeze(batch_x,1)
            batch_y = batch_y.to(device=device, dtype=torch.int64)
            batch_t = batch_t
            output = model.forward(batch_x)
            #output = torch.squeeze(output,1)
            #print('predicted class', torch.argmax(output, axis=1).cpu().numpy())
            #print('actual class', batch_y)
            loss = criterion(output, batch_y)
            val_loss[idx_batch] = loss.item()
            
            pred_probab = nn.Softmax(dim=1)(output)  # Move this TODO
            probs = pred_probab[:, 1]
            
            y_pred_all.append(torch.argmax(output, axis=1).cpu().numpy())
            y_true_all.append(batch_y.cpu().numpy())
            y_score_all.append(probs.detach().cpu().numpy())
            #print('Debug Average Precsion','pred_probab',pred_probab,'probs',probs, 'y_score_all' ,y_score_all)
            
            #''' # Create dataframe and plot true and predicted labels
            num_bs =  len(batch_x)      #256
            num_in = len(batch_x[0,0,0,:])   # 2560
            #print('older t1', num_bs, num_in)
            t1 = np.arange(0,num_in*num_bs,1)
    
            #create a dataframe to visualize randomized data,target and outputs
            df_reconstruct_all = pd.DataFrame(columns=['Channel1', 'Channel2','Channel3','Channel4', 'True_Label', 'Predicted_Label'])
            for i in range(0, len(batch_x)):
                data_re = {'Channel1':  batch_x[i,0,0,:].cpu().numpy(),
                    'Channel2': batch_x[i,0,1,:].cpu().numpy(),
                    'Channel3': batch_x[i,0,2,:].cpu().numpy(),
                    'Channel4': batch_x[i,0,3,:].cpu().numpy(),
                    'True_Label' : np.full((num_in, ), batch_y[i].cpu().numpy()), # 2560 labels
                    'Predicted_Label' : np.full((num_in, ),torch.argmax(output, axis=1)[i].cpu().numpy() ) ,
                    'Time_Label': batch_t[i,:].cpu().numpy()
                    }
                df_reconstruct = pd.DataFrame(data_re)
                df_reconstruct_all = df_reconstruct_all.append(df_reconstruct)
    
            #Calculate detection delay
            dd, s_d = calculate_detection_delay_for_eval(df_reconstruct_all)
            if s_d !=0:
                detection_delay = dd
    
            #Plot all batches
            ''' 
            figure, axs = plt.subplots(nrows=4, ncols=1,figsize=(10,5))
            axs[0].plot( df_reconstruct_all.Time_Label, df_reconstruct_all.Channel1)
            axs[0].set_title('Channel1')
            
            axs[1].plot( df_reconstruct_all.Time_Label, df_reconstruct_all.Channel2)
            axs[1].set_title('Channel2')
    
            axs[2].plot( df_reconstruct_all.Time_Label, df_reconstruct_all.True_Label)
            axs[2].set_title('True Label')
    
            axs[3].plot( df_reconstruct_all.Time_Label, df_reconstruct_all.Predicted_Label )
            axs[3].set_title('Predicted Label')
            plt.show()#'''
            #print(df_reconstruct_all)  # Print the dataframe for each batch
            
    y_pred = np.concatenate(y_pred_all)
    y_true = np.concatenate(y_true_all)
    y_score = np.concatenate(y_score_all)
    print('Check dimensions', y_pred.shape, y_true.shape, y_score.shape)
    
    test_bal_acc = balanced_accuracy_score(y_true, y_pred)
    test_kappa = cohen_kappa_score(y_true, y_pred)
    print(f'Test balanced accuracy: {test_bal_acc:0.3f}')
    print(f'Test Cohen\'s kappa: {test_kappa:0.3f}')
    
    perf = metric(y_true, y_pred)
    f1_score_eval = f1_score(y_true, y_pred, average='binary')
    average_precision = average_precision_score(y_true, y_score)
    
    precision, recall, _ = precision_recall_curve(y_true, y_score)
    auc_precision_recall = auc(recall, precision)
    
    conf_mat = confusion_matrix(y_true, y_pred)
    tn, false_positives, fn, tp = conf_mat.ravel()
    print('conf_mat', conf_mat)
    
    plot_confusion_matrix(conf_mat)
    
    print(f'f1_score_eval: {f1_score_eval:0.3f} \t  average_precision: {average_precision:0.3f} \t'
         f'auc_precision_recall: {auc_precision_recall:0.3f} \t  detection_delay: {detection_delay:0.3f}  \t'
         f'false_positive: {false_positives:0.3f} \t false_negativ: {fn:0.3f}' )
    
    #print('VALID:', np.mean(val_loss), perf, average_precision, f1_score_eval )  
    
    # return loss, accuracy, AP, F1 score,false_+s, delay
    return np.mean(val_loss), perf, average_precision , f1_score_eval, false_positives, detection_delay, fn

def train(model, loader_train, loader_test, optimizer, criterion, n_epochs, 
          patience, device, metric=None):
    """Training function.
    
    Parameters
    ----------
    model : instance of nn.Module
        The model.
    loader_train : instance of Sampler
        The generator of EEG samples the model has to train on.
        It contains n_train samples
    loader_test : instance of Sampler
        The generator of EEG samples the model has to validate on.
        It contains n_val samples. The validation samples are used to
        monitor the training process and to perform early stopping
    optimizer : instance of optimizer
        The optimizer to use for training.
    n_epochs : int
        The maximum of epochs to run.
    patience : int
        The patience parameter, i.e. how long to wait for the
        validation error to go down.
    metric : None | callable
        Metric to use to evaluate performance on the training and
        validation sets. Defaults to balanced accuracy.
        
    Returns
    -------
    best_model : instance of nn.Module
        The model that led to the best prediction on the validation
        dataset.
    history : list of dicts
        Training history (loss, accuracy, etc.)
    """
    #best_valid_loss = np.inf
    #best_model = copy.deepcopy(model)
    waiting = 0
    history = list()
    history_dict = {'epoch': [], 'train_loss': [], 'valid_loss': [],'train_perf':[],'valid_perf':[],'valid_ap':[],
                    'valid_f1':[], 'false_positive':[],'detection_delay':[]}
    
    if metric is None:
        metric = balanced_accuracy_score
        
    print('epoch \t train_loss \t valid_loss \t train_perf \t valid_perf \t valid_ap \t valid_f1')
    print('-------------------------------------------------------------------')
    print('Check number of epochs to run for LOOCV folds after selecting best hyperparam', n_epochs)
    
    for epoch in range(1, n_epochs + 1):
        train_loss, train_perf = _do_train(
            model, loader_train, optimizer, criterion, device, metric=metric)
        
        valid_loss, valid_perf, valid_ap, valid_f1 = _validate(
            model, loader_valid, criterion, device, metric=metric)
        
        history.append(
            {'epoch': epoch, 
             'train_loss': train_loss, 'valid_loss': valid_loss,
             'train_perf': train_perf, 'valid_perf': valid_perf, 'valid_ap': valid_ap,'valid_f1': valid_f1})
        
        print(f'{epoch} \t {train_loss:0.4f} \t {valid_loss:0.4f} '
              f'\t {train_perf:0.4f} \t {valid_perf:0.4f}\t {valid_ap:0.4f}\t {valid_f1:0.4f}')
                                                
        history_dict['epoch'].append(epoch)
        history_dict['train_loss'].append(train_loss)
        history_dict['valid_loss'].append(valid_loss)
        history_dict['train_perf'].append(train_perf)
        history_dict['valid_perf'].append(valid_perf)
        history_dict['valid_ap'].append(valid_ap)
        history_dict['valid_f1'].append(valid_f1)
        '''
        # model saving
        if valid_loss < best_valid_loss:
            print(f'best val loss {best_valid_loss:.4f} -> {valid_loss:.4f}')
            best_valid_loss = valid_loss
            best_model = copy.deepcopy(model)
            waiting = 0
        else:
            waiting += 1

        # model early stopping
        if waiting >= patience:
            print(f'Stop training at epoch {epoch}')
            print(f'Best val loss : {best_valid_loss:.4f}')
            break
        '''
       
    valid_loss, valid_perf, valid_ap, valid_f1, fp, dd, fn = _validate_1epoch(
            model, loader_valid, criterion, device, metric=metric)
    
    print('FINAL EVAL')
    eval_metric = {
        'valid_loss' : valid_loss, 'valid_perf' : valid_perf,'valid_ap' : valid_ap,
        'valid_f1' : valid_f1, 'false_positive' : fp, 'detection_delay' : dd, 'false_negative' : fn
    }
    
    return history_dict, eval_metric

In [ ]:
# Making training test validation split: LOOCV
from sklearn.model_selection import LeaveOneOut
    
def pick_recordings(dataset, subj_rec_num):
    """Pick recordings using subject and recording numbers.
    Parameters
    ----------
    dataset : ConcatDataset
        The dataset to pick recordings from.        
    subj_rec_num : list of tuples
        List of pairs (subj_num, rec_num) to use in split.
        
    Returns
    -------
    pick_ds: The picked recordings.
    remaining_ds : The remaining recordings | None
    """
    pick_idx = list()
    for subj_num, rec_num in subj_rec_num:
        for i, ds in enumerate(dataset.datasets):
            if (ds.subj_num == subj_num) and (ds.rec_num == rec_num):
                pick_idx.append(i)
                
    remaining_idx = np.setdiff1d(range(len(dataset.datasets)), pick_idx)
    print('pick index', pick_idx, 'remaining idx', remaining_idx)

    pick_ds = ConcatDataset([dataset.datasets[i] for i in pick_idx])
    
    if len(remaining_idx) > 0:
        remaining_ds = ConcatDataset([dataset.datasets[i] for i in remaining_idx])
    else:
        remaining_ds = None
        
    print('pick_ids',pick_ds,'remaining_ds' ,remaining_ds) #return test, train
    return pick_ds, remaining_ds

# We seed the random number generators to make our splits reproducible
torch.manual_seed(87)
np.random.seed(87)

# Use 2 recordings for test set
test_recs = [(subj_num, rec_num) for subj_num, rec_num in zip( [patient_id[p_id]] * 10, test_list)]
print('test_recs', test_recs)
test_ds, train_ds = pick_recordings(dataset, test_recs) # Returns  picks_ds, remaining_ds
print(f'Test: {len(test_ds)}')
print(f'train and valid: {len(train_ds)}') #Split into train and validation
'''
def train_test_split(dataset):
    """Split dataset into train and validation keeping n_groups out in validation.
    
    Parameters
    ----------
    dataset : ConcatDataset (The dataset to split.)
    n_groups : int  The number of groups to leave out.
    split_by : 'subj_num' | 'rec_num'
        Property to use to split dataset.
        
    Returns
    -------
    ConcatDataset
        The training data.
    ConcatDataset
        The validation data.
    """
    for fold, (train_idx,valid_idx) in enumerate(LeaveOneOut().split(np.arange(len(dataset.datasets)))):
        
    #splits=KFold(n_splits=k,shuffle=True,random_state=42)
    #groups = [getattr(ds, split_by) for ds in dataset.datasets]
    #print('groups', groups, 'n_groups', n_groups)
    
    #groups = [ds for ds in dataset.datasets]
    #print('groups', groups)
    #train_idx, valid_idx = next(LeaveOneOut().split(np.arange(len(dataset.datasets))))
        print('Check values for train and test', train_idx, valid_idx)
    
    #train_idx, valid_idx = next(LeavePGroupsOut(n_groups).split(X=groups, groups=groups))
    
        train_ds = ConcatDataset([dataset.datasets[i] for i in train_idx])
        valid_ds = ConcatDataset([dataset.datasets[i] for i in valid_idx])
        print(f'Training: {len(train_ds)}')
        print(f'Validation: {len(valid_ds)}')

    return train_ds, valid_ds

# Split remaining recordings into training and validation sets #kfold
#n_subjects_valid = max(1, int(len(train_ds.datasets) * 0.2))
#print('n_subjects_valid', n_subjects_valid,len(train_ds.datasets) )
#train_ds, valid_ds = train_test_split(train_ds, n_subjects_valid, split_by='subj_num') 

#train_ds, valid_ds = train_test_split(train_ds)

#print('Number of examples in each set:')
#print(f'Training: {len(train_ds)}')
#print(f'Validation: {len(valid_ds)}')
#print(f'Test: {len(test_ds)}')'''

In [ ]:
## Computing class weight
from sklearn.utils.class_weight import compute_class_weight

classes_mapping = {0: 'non-ictal', 1: 'ictal'}
train_y = pd.Series([y for _, y,_ in train_ds]).map(classes_mapping)
class_weights = compute_class_weight('balanced', classes=np.unique(train_y), y=train_y)
print('Train ratio',class_weights)
#Visualizing the class distribution, Very imbalanced right now #TODO
#ax_ = train_y.value_counts().plot(kind='barh')
#ax_.set_xlabel('Number of training examples');
#ax_.set_ylabel('Sleep stage');

test_y = pd.Series([y for _, y,_ in test_ds]).map(classes_mapping)
class_weights_test = compute_class_weight('balanced', classes=np.unique(test_y), y=test_y)
print('Test ratio',class_weights_test)

In [ ]:
# TRAIN AND MONITOR NETWORK
from torch.utils.data import DataLoader

# Create dataloaders
train_batch_size = 240  # Important hyperparameter For Now : (720/4) # use 240
valid_batch_size = 240  # Can be made as large as what fits in memory; won't impact performance
num_workers = 0  # Number of processes to use for the data loading process; 0 is the main Python process

loader_train = DataLoader(
    train_ds, batch_size=train_batch_size, shuffle=True, num_workers=num_workers)
loader_valid = DataLoader(
    valid_ds, batch_size=valid_batch_size, shuffle=False, num_workers=num_workers)
loader_test = DataLoader(
    test_ds, batch_size=valid_batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
from torch.nn import CrossEntropyLoss, BCELoss
from torch.optim import Adam
from torch.optim import SGD

# For all fold results
results = {}
optimizer = Adam(model.parameters(), lr=2e-3, weight_decay=0.0001) # TODO : L2 regularizer
#optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
print(torch.Tensor(class_weights))
#criterion = CrossEntropyLoss(weight=torch.Tensor([24.4813,  1]).to(device)) 
criterion = CrossEntropyLoss(weight=torch.Tensor(class_weights).to(device)) #TODO- 
#criterion = CrossEntropyLoss()
#criterion = BCELoss(weight=torch.Tensor(class_weights).to(device))
#summary(model, ( 1, 2, 2560), 180)
print(device)

In [ ]:
# Continue - Add HP, CV on HP, LOO on All, Plotting

In [ ]:
import optuna
import joblib
from sklearn.metrics import balanced_accuracy_score
import csv

def objective(trial, loader_train, loader_valid, class_weights):
    
    # Initialize the best_val_loss value
    best_val_loss = float('Inf')
    history_inner = list()
    history_dict_inner = {'epoch': [], 'train_loss': [], 'valid_loss': [],'train_perf':[],'valid_perf':[],'valid_ap':[],'valid_f1':[]}
   
    joblib.dump(study, 'study.pkl')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Do inner cross vaidation here
    params = {
        'n_filters_1': trial.suggest_int("n_filters_1", 5, 20, step=5), #4
        'n_filters_2': trial.suggest_int("n_filters_2", 5, 15, step=5), #3
        'n_filters_3': trial.suggest_int("n_filters_3", 5, 10, step=5), #2
        'n_filters_4': trial.suggest_int("n_filters_4", 5, 10, step=5), #2
        'n_output_fc1': trial.suggest_int("n_output_fc1", 5, 9, step=2), #3 
        'n_output_fc2': trial.suggest_int("n_output_fc2", 4, 5, step=1), #2
        #'dropout': trial.suggest_float("dropout", 0.2, 0.6, step=0.2),
        'num_epochs': trial.suggest_int("num_epochs", 150, 200, step=50), #2
        'learning_rate': trial.suggest_float('learning_rate', 2e-3, 24e-3, log=True), #0.001-0.024
        #'learning_rate': trial.suggest_loguniform('lgbc__learning_rate', 1e-5, 1e-0),
        #'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
        'optimizer': trial.suggest_categorical("optimizer", ["Adam"])
              }
    # momentum = trial.suggest_float("momentum", 0.0, 1.0)
    #learning_rate_init = trial.suggest_float("learning_rate_init", 1e-5, 1e-3, log=True)
    
    model= IEEGSeizureDetection_2sec(trial,params,n_channels=n_channels, sfreq=sfreq, n_classes=2)
    model= model.to(device)
    model.apply(reset_weights)
    #print('model',model)

    criterion = CrossEntropyLoss(weight=torch.Tensor(class_weights).to(device))
    #criterion = CrossEntropyLoss()
    optimizer = getattr(optim, params['optimizer'])(model.parameters(), lr= params['learning_rate'])
    
    #print('epoch \t train_loss \t valid_loss \t train_perf \t valid_perf \t valid_ap \t valid_f1')
    print('-------------------------------------------------------------------')
    
    start_trial = timer()
    for epoch in range(1, params['num_epochs'] + 1):
        
        train_loss, train_perf = _do_train(
            model, loader_train, optimizer, criterion, device, metric=balanced_accuracy_score)
        
        valid_loss, valid_perf, valid_ap, valid_f1  = _validate(
            model, loader_valid, criterion, device, metric=balanced_accuracy_score)
        
        history_inner.append(
            {'epoch': epoch, 
             'train_loss': train_loss, 'valid_loss': valid_loss,
             'train_perf': train_perf, 'valid_perf': valid_perf, 'valid_ap': valid_ap,'valid_f1': valid_f1})
        
        #print(f'{epoch} \t {train_loss:0.4f} \t {valid_loss:0.4f} '
        #      f'\t {train_perf:0.4f} \t {valid_perf:0.4f}\t {valid_ap:0.4f}\t {valid_f1:0.4f}')
                                                
        history_dict_inner['epoch'].append(epoch)
        history_dict_inner['train_loss'].append(train_loss)
        history_dict_inner['valid_loss'].append(valid_loss)
        history_dict_inner['train_perf'].append(train_perf)
        history_dict_inner['valid_perf'].append(valid_perf)
        history_dict_inner['valid_ap'].append(valid_ap)
        history_dict_inner['valid_f1'].append(valid_f1)
    #return cross validation result
    print('Time taken for 1 fold of HPO CV is', timer() - start_trial  )
    
    print('Return maximum f1 score ',max(history_dict_inner['valid_f1']) )
    print('max valid_acc',max(history_dict_inner['valid_perf']) ,'max_AUPRC',max(history_dict_inner['valid_ap']) )
    return max(history_dict_inner['valid_f1']) # return function you want to maximize

#study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
#if os.path.isfile('study.pkl'):
#    study = joblib.load('study.pkl')
#else:
#    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
    
#study.optimize(objective(loader_train_inner, loader_valid_inner), n_trials=1)

#study.trials_dataframe() - access results  or study.best_value
#model.set_params(**study.best_params)  model.fit(X, y) #train with best params and run test

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, BCELoss
from torch.optim import Adam
from torch.optim import SGD
from torchsummary import summary
import torch.optim as optim

from sklearn.model_selection import LeaveOneOut, KFold
from timeit import default_timer as timer

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#device = 'cuda'
#sfreq = data_raws[0].info['sfreq']  # Sampling frequency
sfreq = 256.0 
#n_channels = data_raws[0].info['nchan']  # Number of channels
n_channels = 4 
#print(f'Using device \'{device}\'.')
#print(model)
best_valid_perf = 0.0
fold_valid_perf = 0.0

# Create dataloaders
train_batch_size = 240  # Important hyperparameter For Now : (720/4) # use 240
valid_batch_size = 240  # Can be made as large as what fits in memory; won't impact performance
num_workers = 0  # Number of processes to use for the data loading process; 0 is the main Python process
#n_epochs = 100
patience = 5
foldperf={}
#F1_std_dev = 0

# File to save final results
out_file = 'LOOCV_results.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)
# Write the headers to the file
writer.writerow(['Best Hyperparameter', 'f1_score Mean', 'f1_score std_dev','Loss', 'Accuracy', 'Avg_Pre',
                 'f1_score', 'false_positives', 'Detection Delay', 'False_negatives'])
of_connection.close()

# File to save intermediate results
'''
out_file_n = 'NestedCV_results.csv'
of_connection_n = open(out_file_n, 'w')
writer = csv.writer(of_connection_n)
# Write the headers to the file
writer.writerow(['Outer Loop', 'Inner Loop', 'Params_1', 'F1max_1', 'Params_2', 'F1max_2', 'Params_3', 'F1max_3', 
                 'Params_4', 'F1max_4', 'Params_5', 'F1max_5', 'Params_6', 'F1max_6',
                 'Params_7', 'F1max_7', 'Params_8', 'F1max_8', 'Params_9', 'F1max_9',
                 'Params_10', 'F1max_10', 'Params_11', 'F1max_11', 'Params_12', 'F1max_12',
                 'Params_13', 'F1max_13', 'Params_14', 'F1max_14', 'Params_15', 'F1max_15',
                 'Params_16', 'F1max_16', 'Params_17', 'F1max_17', 'Params_18', 'F1max_18',
                 'Params_19', 'F1max_19', 'Params_20', 'F1max_20'])
of_connection_n.close()'''

#optimizer = Adam(model.parameters(), lr=2e-3, weight_decay=0) # TODO : add step decay, expo or time based decay
#optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
#criterion = CrossEntropyLoss(weight=torch.Tensor([24.4813,  1]).to(device)) 
#criterion = CrossEntropyLoss(weight=torch.Tensor(class_weights).to(device)) #TODO- 
#criterion = CrossEntropyLoss()
#criterion = nn.BCELoss()
    
#loader_test = DataLoader(
#    test_ds, batch_size=valid_batch_size, shuffle=False, num_workers=num_workers)

#Ref : https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/
       #https://arxiv.org/pdf/1809.09446.pdf

for fold, (train_idx,valid_idx) in enumerate(LeaveOneOut().split(np.arange(len(dataset.datasets)))):
    
    print('LeaveOneOut Cross Validation {}'.format(fold + 1))
    print('Check values for train and test', train_idx, valid_idx)
    
    train_ds = ConcatDataset([dataset.datasets[i] for i in train_idx])
    valid_ds = ConcatDataset([dataset.datasets[i] for i in valid_idx])
    print(f'Training: {len(train_ds)}  Validation: {len(valid_ds)}')
    #print(f'Validation: {len(valid_ds)}')
    
    def objective_cv(trial):
        
        # Ref: https://stackoverflow.com/questions/63224426/how-can-i-cross-validate-by-pytorch-and-optuna
        f1_scores_list = []
        global F1_std_dev
        # InnerFold CV
        inner_cv = KFold(n_splits= int(len(train_ds.datasets)/2), shuffle=False)
        print('Number of folds in inner cv',int(len(dataset.datasets)/2),',trial:',trial)
        for (k_fold, (train_index_inner, test_index_inner)) in enumerate(inner_cv.split(range(len(train_ds.datasets)))):
        
            print('HPO with Cross Validation {}'.format(k_fold + 1))
            print('Check values for inner train and test', train_index_inner, test_index_inner)
    
            train_ds_inner = ConcatDataset([train_ds.datasets[i] for i in train_index_inner])
            valid_ds_inner = ConcatDataset([train_ds.datasets[i] for i in test_index_inner])
            print(f'Training_inner_cv: {len(train_ds_inner)}  Validation_inner_cv: {len(valid_ds_inner)}')
            
            #Check weigh ratios for train and valid
            classes_mapping = {0: 'non-ictal', 1: 'ictal'}
            train_y_inner = pd.Series([y for _, y,_ in train_ds_inner]).map(classes_mapping)
            valid_y_inner = pd.Series([y for _, y,_ in valid_ds_inner]).map(classes_mapping)
            class_weights_inner = compute_class_weight('balanced', classes=np.unique(train_y_inner), y=train_y_inner) 
            class_weights_valid_inner = compute_class_weight('balanced', classes=np.unique(valid_y_inner), y=valid_y_inner)
            print('Weights ratio: Train',class_weights_inner,'Test',class_weights_valid_inner)
    
            #Dataloaders for train and valid
            loader_train_inner = DataLoader(train_ds_inner, batch_size=train_batch_size, shuffle=True, num_workers=num_workers)
            loader_valid_inner = DataLoader(valid_ds_inner, batch_size=valid_batch_size, shuffle=False, num_workers=num_workers)
        
            f1_score_max = objective(trial, loader_train_inner, loader_valid_inner,class_weights_inner)
            print('Maximum f1 score for innerfold ',k_fold,'is:', f1_score_max)
            #TODO : take the mean of all cross validation scores
            f1_scores_list.append(f1_score_max) #List length = number of folds
            print('Length of f1_score_list', len(f1_scores_list), f1_scores_list)
            
        print('Mean f1_score for this trial is', np.mean(f1_scores_list),'with standard deviation',np.std(f1_scores_list))  
        F1_std_dev =  np.std(f1_scores_list)
        #return np.mean(scores)   
        return np.mean(f1_scores_list)
   
    start = timer()
    #Evaluate Hyperparameters and return best ones
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
    study.optimize(objective_cv, n_trials=100)
    print('Time taken for one study of optimization is',timer() - start)

    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    
    trial_df = study.trials_dataframe()
    trial_df.to_csv('all_trials.csv', encoding='utf-8') # all trials, for all outer folds
    
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))
    #print("  Dataframe from  all trials: ", trial_df)
    trial = study.best_trial
    print("Best trial:",trial)
    print("  Value: ", study.best_trial.value) # Returns the best f1_score (average across cross validations)
    
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    print('Using Best parameters from study of one LOOCV')
    print('study.best_params',study.best_params)
    #print('trial.params',trial.params)
    #print('study.best_trial.params',study.best_trial.params)   
    
    #Check weigh ratios for train and valid
    classes_mapping = {0: 'non-ictal', 1: 'ictal'}
    train_y = pd.Series([y for _, y,_ in train_ds]).map(classes_mapping)
    valid_y = pd.Series([y for _, y,_ in valid_ds]).map(classes_mapping)
    class_weights = compute_class_weight('balanced', classes=np.unique(train_y), y=train_y) 
    class_weights_valid = compute_class_weight('balanced', classes=np.unique(valid_y), y=valid_y)
    print('Weights ratio: Train',class_weights,'Test',class_weights_valid)

    #Dataloaders for train and valid
    loader_train = DataLoader(train_ds, batch_size=train_batch_size, shuffle=True, num_workers=num_workers)
    loader_valid = DataLoader(valid_ds, batch_size=valid_batch_size, shuffle=False, num_workers=num_workers)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model= IEEGSeizureDetection_2sec(trial,study.best_params,n_channels=n_channels, sfreq=sfreq, n_classes=2)
    model.to(device)
    print('BEST MODEL SELECTED AFTER HYPERPARAMETER OPTIMIZATION IS', model)
    #model = model.to(device)
    model.apply(reset_weights)
    #print('Values to select from best param',study.best_params['num_epochs'],study.best_params['learning_rate'],study.best_params['optimizer'])

    criterion = CrossEntropyLoss(weight=torch.Tensor(class_weights).to(device))
    #criterion = BCELoss(weight=torch.Tensor(class_weights).to(device), reduction='none')
    optimizer = Adam(model.parameters(), lr=study.best_params['learning_rate'], weight_decay=0)
    #optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])
    
    #history = train(
    #model, loader_train, loader_valid , optimizer, criterion, n_epochs, patience, 
    #device, metric=balanced_accuracy_score)
    
    history, eval_metric = train(
    model, loader_train, loader_valid , optimizer, criterion, study.best_params['num_epochs'], patience, 
    device, metric=balanced_accuracy_score)
    
    # # Write to the final csv file 
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([study.best_params, study.best_trial.value, F1_std_dev,eval_metric['valid_loss'],
                     eval_metric['valid_perf'], eval_metric['valid_ap'], eval_metric['valid_f1'],
                    eval_metric['false_positive'],eval_metric['detection_delay'] ,eval_metric['false_negative']])
    of_connection.close()

    foldperf['fold{}'.format(fold+1)] = history  
    
    fold_valid_perf = np.mean(foldperf['fold{}'.format(fold+1)]['valid_perf'][study.best_params['num_epochs']-10:study.best_params['num_epochs']-1])*100.0
    if fold_valid_perf > best_valid_perf:
        best_valid_perf = fold_valid_perf
        print('best_valid_perf', best_valid_perf)
        print('Training process has finished. Saving trained model for', fold)   
        save_path = f'./saved_model/model-{patient_id[p_id]}-fold-{fold}.pth'
        torch.save(model.state_dict(), save_path)
    else:
        print('Not the best_valid_perf',fold_valid_perf)
    
    print('Evaluation results at the end of test set of LOOCV',eval_metric,'for fold',fold)
    #break

In [ ]:
params, values

In [ ]:
m = nn.Sigmoid()
loss = BCELoss()
input = torch.randn(3)
target = torch.empty(3).random_(2)
print('input',np.shape(input),'Target',np.shape(target))
output = loss(m(input), target)


In [ ]:
# function which returns dataframe of all evaluation for LOOCV
#Evaluation metrics:  Validation Loss, Validation Accuracy, Average Precision, F1 score, False Positive, Detection delay

In [ ]:
avg_train_loss = 0.0
avg_valid_loss = 0.0
avg_train_perf = 0.0
avg_valid_perf = 0.0
avg_train_loss_list = []
avg_valid_loss_list = []
avg_train_perf_list = []
avg_valid_perf_list = []
#print(foldperf['fold1']['train_loss'][n_epochs-1])
for fold in range(0,len(train_ds.datasets)+1):
    #print(foldperf['fold{}'.format(fold+1)])
    #print('fold{}'.format(fold+1))
    avg_train_loss += np.mean(foldperf['fold{}'.format(fold+1)]['train_loss'][n_epochs-10:n_epochs-1])
    avg_valid_loss += np.mean(foldperf['fold{}'.format(fold+1)]['valid_loss'][n_epochs-10:n_epochs-1])
    avg_train_perf += np.mean(foldperf['fold{}'.format(fold+1)]['train_perf'][n_epochs-10:n_epochs-1])
    avg_valid_perf += np.mean(foldperf['fold{}'.format(fold+1)]['valid_perf'][n_epochs-10:n_epochs-1])
    avg_train_loss_list.append(np.mean(foldperf['fold{}'.format(fold+1)]['train_loss'][n_epochs-10:n_epochs-1]))
    avg_valid_loss_list.append(np.mean(foldperf['fold{}'.format(fold+1)]['valid_loss'][n_epochs-10:n_epochs-1]))
    avg_train_perf_list.append(np.mean(foldperf['fold{}'.format(fold+1)]['train_perf'][n_epochs-10:n_epochs-1]))
    avg_valid_perf_list.append(np.mean(foldperf['fold{}'.format(fold+1)]['valid_perf'][n_epochs-10:n_epochs-1]))
print(f'Average Train Loss: {avg_train_loss/len(train_ds.datasets)} ')
print(f'Average Valid Loss: {avg_valid_loss/len(train_ds.datasets)} ')
print(f'Average Train Perf: {avg_train_perf/len(train_ds.datasets)} ')
print(f'Average Valid Perf: {avg_valid_perf/len(train_ds.datasets)} ')

#Plot evluation metric for all folds
cv_folds = list(range(len(train_ds.datasets)+1)) 
y = avg_valid_perf_list
print('y-4',y)
plt.xlabel("LOO folds")
plt.ylabel("Eval")
plt.title("LOO CV")
#for i in range(len(y)):
#    plt.plot(cv_folds,[pt[i] for pt in y],label = 'Metric%s'%i)
plt.plot(cv_folds,y,label = 'Training performance')
plt.legend()
plt.show();

In [ ]:
n_epochs = 150
patience = 5

history = train(
    model, loader_train, loader_valid , optimizer, criterion, n_epochs, patience, 
    device, metric=balanced_accuracy_score)


In [ ]:
#Save the model with best val and plot/use that
# Visualizing the learning curves
#print(foldperf['fold3'])
history_df = pd.DataFrame(foldperf['fold5'])
print(history_df)
ax1 = history_df.plot(x='epoch', y=['train_loss', 'valid_loss'], marker='o')
ax1.set_ylabel('Loss')
ax2 = history_df.plot(x='epoch', y=['train_perf', 'valid_perf'], marker='o')
ax2.set_ylabel('balanced_accuracy_score')

In [ ]:
history_df = pd.DataFrame(history)
ax1 = history_df.plot(x='epoch', y=['train_loss', 'valid_loss'], marker='o')
ax1.set_ylabel('Loss')
ax2 = history_df.plot(x='epoch', y=['train_perf', 'valid_perf'], marker='o')
ax2.set_ylabel('balanced_accuracy_score')

In [ ]:
print(save_path)

In [ ]:
#Compute test performance
torch.set_printoptions(precision=2)
#best_model.eval()
model.load_state_dict(torch.load(save_path))
model.eval()

y_pred_all, y_true_all = list(), list()
y_score_all = list()
for batch_x, batch_y,batch_t in loader_test:
    batch_x = batch_x.to(device=device, dtype=torch.float32)
    batch_y = batch_y.to(device=device, dtype=torch.int64)
    batch_t = batch_t                                       #([240, 512])
    
    #print('Check X',batch_x, batch_x.shape )
    #print('Check Y',batch_y, batch_y.shape )
    #print('Check time delay',batch_t, batch_t.shape )
    output = model.forward(batch_x)
    pred_probab = nn.Softmax(dim=1)(output)  # Move this TODO
    probs = pred_probab[:, 1]             #Predicting probability
    torch.set_printoptions(precision=2)
    #print('output of neural network',pred_probab, pred_probab.shape, 'probs',probs)
    print('target class', batch_y.cpu().numpy())
    print('predicted class', torch.argmax(output, axis=1).cpu().numpy())
    
    y_pred_all.append(torch.argmax(output, axis=1).cpu().numpy())
    y_true_all.append(batch_y.cpu().numpy())
    y_score_all.append(probs.detach().cpu().numpy())
    
    #''' # Create dataframe and plot true and predicted labels
    num_bs =  len(batch_x)      #256
    num_in = len(batch_x[0,0,0,:])   # 2560
    print('older t1', num_bs, num_in)
    t1 = np.arange(0,num_in*num_bs,1)
    #t1 = batch_t
    
    #create a dataframe to visualize randomized data,target and outputs
    df_reconstruct_all = pd.DataFrame(columns=['Channel1', 'Channel2','Channel3','Channel4', 'True_Label', 'Predicted_Label'])
    for i in range(0, len(batch_x)):
        data_re = {'Channel1':  batch_x[i,0,0,:].cpu().numpy(),
            'Channel2': batch_x[i,0,1,:].cpu().numpy(),
            'Channel3': batch_x[i,0,2,:].cpu().numpy(),
            'Channel4': batch_x[i,0,3,:].cpu().numpy(),
            'True_Label' : np.full((num_in, ), batch_y[i].cpu().numpy()), # 2560 labels
            'Predicted_Label' : np.full((num_in, ),torch.argmax(output, axis=1)[i].cpu().numpy() ) ,
            'Time_Label': batch_t[i,:].cpu().numpy()
            }
        df_reconstruct = pd.DataFrame(data_re)
        df_reconstruct_all = df_reconstruct_all.append(df_reconstruct)
    
    #Calculate detection delay
    calculate_detection_delay_for_eval(df_reconstruct_all)
    
    #Plot all batches
    #''' 
    print('BATCH_NUM',i)
    figure, axs = plt.subplots(nrows=4, ncols=1,figsize=(10,5))
    axs[0].plot( df_reconstruct_all.Time_Label, df_reconstruct_all.Channel1)
    axs[0].set_title('Channel1')
    
    axs[1].plot( df_reconstruct_all.Time_Label, df_reconstruct_all.Channel2)
    axs[1].set_title('Channel2')
    
    axs[2].plot( df_reconstruct_all.Time_Label, df_reconstruct_all.True_Label)
    axs[2].set_title('True Label')
    
    axs[3].plot( df_reconstruct_all.Time_Label, df_reconstruct_all.Predicted_Label )
    axs[3].set_title('Predicted Label')
    plt.show()
    #axs[0].set_xlabel('distance (m)')
    #axs[0].set_ylabel('Damped oscillation')'''
    '''
    plt.figure()
    plt.subplot(411)
    
    plt.plot(t1, df_reconstruct_all.Channel1 )
    
    plt.subplot(412)
    plt.plot(t1, df_reconstruct_all.Channel2)
    
    plt.subplot(413)
    plt.plot(t1, df_reconstruct_all.True_Label )
    
    plt.subplot(414)
    plt.plot(t1, df_reconstruct_all.Predicted_Label )
    plt.show()
    '''
    #print(df_reconstruct_all)  # Preint the dataframe for each batch

y_pred = np.concatenate(y_pred_all)
y_true = np.concatenate(y_true_all)
y_score = np.concatenate(y_score_all)

test_bal_acc = balanced_accuracy_score(y_true, y_pred)
test_kappa = cohen_kappa_score(y_true, y_pred)
print(f'Test balanced accuracy: {test_bal_acc:0.3f}')
print(f'Test Cohen\'s kappa: {test_kappa:0.3f}')
print('y_score', y_score.shape)


In [ ]:
# VISUALIZE THE RESULTS
classes_mapping = {0: 'non-ictal', 1: 'ictal'}
y_train = pd.Series([y for _, y,_ in train_ds]).map(classes_mapping)

from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(conf_mat, classes_mapping):
    ticks = list(classes_mapping.keys())
    tick_labels = classes_mapping.values()

    fig, ax = plt.subplots(figsize=(6, 6))
    im = ax.imshow(conf_mat, cmap='Reds')
    ax.set_yticks(ticks)
    ax.set_yticklabels(tick_labels)
    ax.set_xticks(ticks)
    ax.set_xticklabels(tick_labels)
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')
    ax.set_title('Confusion matrix')

    for i in range(len(ticks)):
        for j in range(len(ticks)):
            text = ax.text(
                j, i, conf_mat[i, j], ha='center', va='center', color='k')

    fig.colorbar(im, ax=ax, fraction=0.05, label='# examples')
    fig.tight_layout()
    
    return fig, ax

conf_mat = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = conf_mat.ravel()
print('conf_mat', conf_mat)
print('TN:',tn,'FP',fp, 'FN', fn, 'TP', tp)
precision_cm = tp/(tp+fp)
recall_cm = tp/(tp+fn)
accuracy_cm = (tn +tp)/(tn+fp+fn+tp)
F1_cm = 2*(precision_cm * recall_cm)/(precision_cm + recall_cm)

tpr = tp/(tp+fn)
fnr = fn/(tp+fn)
tnr = tn/(tn+fp)
fpr = fp/(tn+fp)

print('Precision', precision_cm, '\nRecall',recall_cm, '\naccuracy',accuracy_cm, '\nF1', F1_cm)
print('TPR:',tpr,'TNR', tnr,'\nFNR',fnr,  'FPR', fpr)
plot_confusion_matrix(conf_mat, classes_mapping)



In [ ]:
## Evaluation Metrics - PR curve, AUC, F1, AP
# Precision Recall
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve, roc_curve
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score, average_precision_score
from matplotlib import pyplot
from sklearn.metrics import auc
from sklearn.metrics import f1_score, precision_score, recall_score, fbeta_score
from sklearn.metrics import cohen_kappa_score

# calculate roc and roc-auc
print('check size', y_true.shape, y_score.shape)
fpr, tpr, thr = roc_curve(y_true, y_score)
roc_auc = roc_auc_score(y_true, y_score)
print('roc auc %.3f' % roc_auc)

#Calculate F1, precision,recall
F1_score = f1_score(y_true, y_pred, average='binary')
F2_score = fbeta_score(y_true, y_pred, average='weighted', beta=2)
Precision_score = precision_score(y_true, y_pred, average="micro")
Recall_score = recall_score(y_true, y_pred, average="micro")
test_kappa = cohen_kappa_score(y_true, y_pred)
print('Test Cohen\'s kappa: %.3f' % test_kappa)
print('F1-Measure: %.3f' % F1_score)
print('F2-Measure: %.3f' % F2_score)
#rint('Precision-Measure: %.3f' % Precision_score)
#rint('Recall-Measure: %.3f' % Recall_score)

# plot roc curves
#plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, linestyle='--',color='green', label='ROC')
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show();

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

#print('y_true', y_true, y_true.shape, 'y_score', y_score, y_score.shape)
precision, recall, _ = precision_recall_curve(y_true, y_score)
average_precision = average_precision_score(y_true, y_score)
auc_precision_recall = auc(recall, precision)

print('Precision from PRcurve',np.mean(precision), 'Recall from PRcurve',np.mean(recall))
print('Average Precision', average_precision)
print('Areas under Precision Recall curve', auc_precision_recall)

display = PrecisionRecallDisplay(recall=recall,precision=precision,average_precision=average_precision, estimator_name=None)
display.plot()
_ = display.ax_.set_title("Precision Recall Curve")

In [ ]:
#Create ONNX model
